<a href="https://colab.research.google.com/github/alozy15/A_B-Testing/blob/master/EOM_close.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import datetime
from datetime import date
import holidays
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd
import warnings
import seaborn as sns
#import pygsheets 
import itertools
import functools
warnings.filterwarnings('ignore')
print( os.getcwd())

import google.auth
import google.cloud as gcp
from google_auth_oauthlib import flow
from google.cloud import storage
from google.cloud import bigquery
from oauth2client import file 

from pathlib import Path



from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)



/content


# Queries

In [ ]:
#Op1 BQ Data Pull

project_id = 'rv-mt-data-prod'

op1 = pd.io.gbq.read_gbq("""
SELECT *
FROM
(
(
SELECT
m.delivery_month_id,
o.sales_order_id,
ol.sales_order_line_item_id,
o.advertiser_nm,
TRIM(o.sales_order_nm) as sales_order_nm,

-- Case to identify Programmatic Guaranteed order types
CASE WHEN o.sales_order_type_desc like'%Programmatic%'THEN 'Custom Programmatic Guaranteed'
     ELSE o.sales_order_type_desc
END AS sales_order_type_desc,

o.order_status_desc,

-- Case to categorize the order status into different stages i.e Proposal, deleted etc.
CASE WHEN o.order_status_desc in ('proposal_creation','proposal_approvals','preliminary_order_approvals')THEN 'Proposal'
     WHEN o.order_status_desc in ('client_proposal_approval','final_order_creation')THEN 'Pending'
     WHEN o.order_status_desc in ('proposed_order_modification')THEN 'Pending/Modification'
     WHEN o.order_status_desc in ('closed_lost','delete')THEN 'Deleted'
     ELSE 'Booked' 
END AS aggregate_order_status,

CAST(o.contract_close_dt as date) as contract_close_date,
o.primary_salesperson_nm,
o.owner_nm,
o.order_primary_team_nm,
o.agency_nm,
ol.line_item_start_dt,
ol.line_item_end_dt,
ol.sales_order_line_item_nm,
p.product_id,
p.product_nm,
p.product_type_desc,

-- Case to categorize product based on the GEO embedded in their name
CASE WHEN REGEXP_CONTAINS(LOWER(p.product_nm),r'^au\||^emea\||^uk\||^fr\||^sg\||^asia\||^jp\||^kr\||^in\||^cn\||^us')THEN 'Intl'
     WHEN REGEXP_CONTAINS(LOWER(p.product_nm), r'\|download country\||apac\||asia|australia|china|emea\||europe|france|germany|latin america|shunt row\||taiwan|uk\||united kingdom|au\|')THEN 'Intl'
     WHEN REGEXP_CONTAINS(LOWER(p.product_nm),r'^a\||^a_')THEN 'US'
ELSE 'Unmapped'
END AS Prod_Geo,

ol.unit_type_desc,
ol.makegood_ind,
ol.booking_type_desc,
p.rev_allocation_billing_id_desc as gl_code,
ol.floor_price_amt_s4/10000 as floor_price_amt,-- operative stores some values as 1x^10000 so we have to divide by 10000 to get the actual value
o.total_order_value_amt_s4/10000 as total_order_value_amt,
o.ecpm_amt_s4/10000 as ecpm_amt,
ol.cost_type_desc,
ol.net_unit_cost_amt_s4/10000 as net_unit_cost_amt,
SUM(m.booked_qty_distributed_s4)/10000 as Impressions,
SUM(m.net_cost_distributed_amt_s4)/10000 as Revenue,

o.sales_order_id,
o.sales_order_nm,
LEFT(CAST(m.delivery_month_id AS string),4) AS year,
RIGHT(CAST(m.delivery_month_id AS string),2) AS month,

CASE WHEN REGEXP_CONTAINS(p.product_nm,r"CNET") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"RV Games") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Digital Lifestyle") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Tech") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Other") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"ZDNet|ZDNET") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"GameSpot|Gamespot|gamespot.com") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"TV Guide|TV|Guide.com|TVGuide|TV&Guide") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"B2B|Business|Business Technology") THEN"M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"ThePointsGuy|The Points Guy") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"NextAdvisor") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"GameFAQs") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"B2B|Business|Business Technology") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Bankrate") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Metacritic") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Lonely Planet|LonelyPlanet|LP") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Download") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Comic Vine|Comicvine") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Chowhound|Chow") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"EDU") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Giant Bomb|Giantbomb|GiantBomb") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Youtube") THEN "M&C"				
ELSE "Other" END AS product_type,



FROM `rv-mt-data-prod.dw_vw.op_sales_order_cmg` o
INNER JOIN `rv-mt-data-prod.dw_vw.op_sales_order_line_items_cmg`  ol on ol.sales_order_id = o.sales_order_id
INNER JOIN `rv-mt-data-prod.dw_vw.op_product_cmg` p on p.product_id = ol.product_id
INNER JOIN `rv-mt-data-prod.dw_vw.op_orderline_month_cmg` m on m.sales_order_line_item_id= ol.sales_order_line_item_id
LEFT JOIN `rv-mt-data-prod.dw_vw.op_product_custom_data_cmg` pc on p.product_id = pc.product_id

WHERE
m.delivery_month_id >=((extract( Year from current_date)-4)*100+1)
AND ol.parent_line_item_id IS NULL
AND ol.line_item_status_desc <>'deleted'
AND LOWER(ol.sales_order_line_item_nm) NOT like ('%delet%')
AND LOWER(o.sales_order_nm) NOT LIKE ('%delete%')
AND o.order_status_desc NOT IN ('closed_lost','deleted')
AND o.advertiser_nm NOT IN ('CBS Interactive','Test')
AND o.sales_order_type_desc NOT IN ('Internal Trafficking','Barter','MMOD Internal','RevShare','Network','CCS','Red Ventures RevShare','Leads','Addressable','Ad System Test (Admin Use Only) ')
AND o.order_primary_team_nm NOT IN ('CBSi Deleted Orders','CBSI SG - B2B')


AND (
    REGEXP_CONTAINS(LOWER(ol.product_nm),r'gamespot|gamefaq|comicvine|giant.?bomb|cbsi games|games|twitch|business technology|\|download\||download country|smartplanet|\|direct\||zdnet|b2b|tech.*republic|business 360|cnet|download|digital lifestyle|tv guide|\|tv\||chow|tv shows on dvd|metacritic|\|cbsi\|games\||metro.*lyrics|ongamers|\|tech\||gamerankings|surround sound|\|business|\|media\||media group|cbsi cmg network|\|network b2b\||\|network b2c\||\|b2c\||\|network\||\|cbsi tech\||\|network run of tech|enterprise tech|cbsi consumer network|cbsi business network|mp3|b2c|ccs|b2b|b2d|tv.com|tech network|business network|consumer network|digital brands|enterprise tech|consumer tech|music network|run of tech|a\|\|guide|\|rv|\|studio 61|\|digital tech lifestyle|emea\|\|network\.us\||uk|cbsi\|')
    
    AND NOT REGEXP_CONTAINS(LOWER(ol.product_nm),r'cbs sports|can sports network|au ten\||\|cbs interactive entertainment\||\|cbs news\||\|digital brands\|last\.fm|last.?fm|cbs\.com')
OR 
(
    REGEXP_CONTAINS(LOWER(ol.product_nm),r'cbsi network') AND ol.line_item_start_dt < '2021-01-01'
))

---exclude INTL products, need to make a INTL-only Template
AND NOT REGEXP_CONTAINS(LOWER(p.product_nm),r'^au\||^emea\||^uk\||^fr\||^sg\||^asia\||^jp\||^kr\||^in\||^cn\||^us')
AND NOT REGEXP_CONTAINS(LOWER(p.product_nm), r'\|download country\||apac\||asia|australia|china|emea\||europe|france|germany|latin america|shunt row\||taiwan|uk\||united kingdom|au\|')
AND  REGEXP_CONTAINS(LOWER(p.product_nm),r'^a\||^a_')

--excludes non-MAS products
AND NOT (REGEXP_CONTAINS(LOWER(p.product_nm), r'\|revshare\||\|search revenue|\|search\.com\||commerce\||licensing\|')) 

AND o.sales_order_id NOT in (25546,26632,27832,30605,31039,31615,33611,34225,34331,34859,34863,35247,35410,37671,37677,37741,37796,37799,38073,38222,38272,38328,38640,38723,40272,40499,41859,43595,43851,44337,44955,44987,44990,45002,45003,45015,45025,46183,46331,46380,46385,46879,47305,47434,47915,48518,49551,49569,49573,49578,49581,50107,50224,50526,50677,50985,50987,51060,51989,52477,52933,53044,53240,53273,53659,53661,53662,53663,53665,53666,53669,53670,53673,53834,54356,54672,54900,54902,54913,56039,56040,56637,56853,57088,57093,57228,58371,58540,58967,59014,59094,59174,59223,59519,59787,59876,60003,61152,62564,62566,62722,62914,63190,64674,64766,64916,66487,67570,67657,67725,67874,67892,67894,67895,68392,68551,69106,69233,69314,69361,69670,69674,69676,69679,69681,69683,69685,69690,69691,69692,69735,69737,69739,69808,69828,69829,69830,69833,69834,69835,69836,69840,69841,69845,69846,69849,69852,69872,69873,69874,69960,70087,70089,70095,70096,70097,70217,70342,70348,70380,70386,70545,70719,70829,71360,71443,71448,71498,71559,71731,71874,71911,72008,72205,72206,72207,72208,72211,72219,72323,72356,72418,72569,72576,72656,72714,72727,72807,72848,72887,72896,72965,73026,73053,73098,73103,73106,73158,73247,73776,72264,73076,63185,73327,73900,72094,62363,64409,74202,73273,73507,71584,74182,72988,71733,73090,71827,73843,73654,72114,73090,73090)
#AND delivery_month_id = 202301
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,32,33

ORDER BY 1,2,3
)

-- Combining the results above with the results below  
Union All


(
SELECT
m.delivery_month_id,
o.sales_order_id,
ol.sales_order_line_item_id,
o.advertiser_nm,
TRIM(o.sales_order_nm) AS sales_order_nm,

sales_order_type_desc,

o.order_status_desc,

-- Case to categorize the order status into different stages i.e Proposal, deleted etc.
CASE WHEN o.order_status_desc in ('proposal_creation','proposal_approvals','preliminary_order_approvals')THEN 'Proposal'
     WHEN o.order_status_desc in ('client_proposal_approval','final_order_creation')THEN 'Pending'
     WHEN o.order_status_desc in ('proposed_order_modification')THEN 'Pending/Modification'
     WHEN o.order_status_desc in ('closed_lost','delete')THEN 'Deleted'
     ELSE 'Booked' END AS aggregate_order_status,

CAST(o.contract_close_dt as date) as contract_close_date,
o.primary_salesperson_nm,
o.owner_nm,
o.order_primary_team_nm,
o.agency_nm,
CAST( ol.line_item_start_dt as date) as line_item_start_dt,
CAST( ol.line_item_end_dt as date) as line_item_end_dt,
ol.sales_order_line_item_nm,
p.product_id,
p.product_nm,
p.product_type_desc,

-- Case to categorize product based on the GEO embedded in their name
CASE WHEN REGEXP_CONTAINS(LOWER(p.product_nm),r'^cm\|au\||^cm\|emea\||^cm\|uk\||^cm\|fr\||^cm\|sg\||^cm\|asia\||^cm\|jp\||^cm\|kr\||^cm\|in\||^cm\|cn\||^cm\|us') THEN 'Intl'
     WHEN REGEXP_CONTAINS(LOWER(p.product_nm), r'\|download country\||apac\||asia|australia|china|emea\||europe|france|germany|latin america|shunt row\||taiwan|uk\||united kingdom|au\|')THEN 'Intl'
     WHEN REGEXP_CONTAINS(LOWER(p.product_nm),r'^cm\|a\||^a\|')THEN 'US'
     ELSE 'Unmapped'
END AS Prod_Geo,

ol.unit_type_desc,
ol.makegood_ind,
ol.booking_type_desc,
p.rev_allocation_billing_id_desc as gl_code,
ol.floor_price_amt_s4/10000 as floor_price_amt,-- operative stores some values as 1x^10000 so we have to divide by 10000 to get the actual value
o.total_order_value_amt_s4/10000 as total_order_value_amt,
o.ecpm_amt_s4/10000 as ecpm_amt,    
ol.cost_type_desc,
ol.net_unit_cost_amt_s4/10000 as net_unit_cost_amt,
SUM(m.booked_qty_distributed_s4)/10000 as Impressions,
SUM(m.net_cost_distributed_amt_s4)/10000 as Revenue,

o.sales_order_id,
o.sales_order_nm,

LEFT(CAST(m.delivery_month_id AS string),4) AS year,
RIGHT(CAST(m.delivery_month_id AS string),2) AS month,

CASE WHEN REGEXP_CONTAINS(p.product_nm,r"CNET") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"RV Games") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Digital Lifestyle") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Tech") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Other") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"ZDNet|ZDNET") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"GameSpot|Gamespot|gamespot.com") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"TV Guide|TV|Guide.com|TVGuide|TV&Guide") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"B2B|Business|Business Technology") THEN"M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"ThePointsGuy|The Points Guy") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"NextAdvisor") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"GameFAQs") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"B2B|Business|Business Technology") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Bankrate") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Metacritic") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Lonely Planet|LonelyPlanet|LP") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Download") THEN "M&C"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Comic Vine|Comicvine") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Chowhound|Chow") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"EDU") THEN "RVMN"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Giant Bomb|Giantbomb|GiantBomb") THEN "G&E"				
WHEN REGEXP_CONTAINS(p.product_nm,r"Youtube") THEN "M&C"				
ELSE "Other" END AS product_type,

FROM `rv-mt-data-prod.dw_vw.op_sales_order` AS o
INNER JOIN `rv-mt-data-prod.dw_vw.op_sales_order_line_items`  ol ON ol.sales_order_id = o.sales_order_id
INNER JOIN `rv-mt-data-prod.dw_vw.op_product` p ON p.product_id = ol.product_id
INNER JOIN `rv-mt-data-prod.dw_vw.op_orderline_month` m ON m.sales_order_line_item_id= ol.sales_order_line_item_id

WHERE
m.delivery_month_id >=((extract( Year from current_date)-4)*100+1)
AND ol.parent_line_item_id is null
AND ol.line_item_status_desc <>'deleted'
AND LOWER(ol.sales_order_line_item_nm) NOT like ('%delet%')
AND LOWER(o.sales_order_nm) NOT like ('%delet%')
AND o.order_status_desc NOT IN ('closed_lost','deleted')
AND o.advertiser_nm NOT IN ('RV Test Advertiser')
AND o.sales_order_type_desc NOT IN ('Licensing Fees','Affiliate Leads','RevShare','Open Exchange-Programmatic','Recurly Subscription','Ad System Test (Admin Use Only) ')

---exclude INTL products, (eventually they should all be deprecated. - ignore.(2/28/22)
AND NOT REGEXP_CONTAINS(LOWER(p.product_nm),r'^cm\|au\||^cm\|emea\||^cm\|uk\||^cm\|fr\||^cm\|sg\||^cm\|asia\||^cm\|jp\||^cm\|kr\||^cm\|in\||^cm\|cn\||^cm\|us')
AND NOT REGEXP_CONTAINS(LOWER(p.product_nm), r'\|download country\||apac\||asia|australia|china|emea\||europe|france|germany|latin america|shunt row\||taiwan|uk\||united kingdom|au\|')
AND REGEXP_CONTAINS(LOWER(p.product_nm),r'^cm\|a\||^a\|')

--PG Order for EOM rev rec
AND o.sales_order_id NOT IN (80709,82340)
#AND delivery_month_id = 202301

GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,32,33

ORDER BY 1,2,3

)

ORDER BY 1,2,3
)

WHERE year >= '2023' AND product_type <> "G&E" AND sales_order_type_desc IN ('Media', 'Custom Programmatic Guaranteed')
ORDER BY month, year

""", project_id= project_id)

op1.month = op1.month.astype(int)

op1.loc[op1['month'].isin([1,2,3]), 'quarter_nbr'] = 1
op1.loc[op1['month'].isin([4,5,6]), 'quarter_nbr'] = 2
op1.loc[op1['month'].isin([7,8,9]), 'quarter_nbr'] = 3
op1.loc[op1['month'].isin([10,11,12]), 'quarter_nbr'] = 4
op1['month_nbr'] = op1['month']

In [ ]:
project_id = 'rv-mt-data-prod'

op_fi = pd.io.gbq.read_gbq("""


DECLARE year_start DATE;
DECLARE prev_month_end DATE;
DECLARE this_month_start DATE;
-- SET prev_month_start = (SELECT DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 1 Month));
-- SET prev_month_end = (SELECT DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 1 day));
-- SET this_month_start = (SELECT DATE_TRUNC(current_date(), month));
SET year_start = "2021-10-01";
SET prev_month_end = "2023-04-30";
SET this_month_start = "2023-05-01";

with pg_lines_by_month AS (
  SELECT
    go.sfa_order_id,
    -- go.line_item_nm,
    go.sfa_order_line_nbr,
    DATE_TRUNC(gd.day_dt, month) AS delivered_month,
    SUM(impressions) ad_imps,
    SUM(clicks) ad_clicks,
    CAST(SUM(impressions * final_cpm_amt_s4)/10000000 AS NUMERIC) ad_revenue
  FROM erp_vw.erp_delivery_day gd
  JOIN dw_vw.gam_order_line_item_dim go USING(order_line_item_id)
  JOIN dw_vw.op_sales_order so ON go.sfa_order_id = so.sales_order_id
    AND so.sales_order_type_desc = 'Custom Programmatic Guaranteed'
    AND so.order_status_desc <> 'deleted'
  LEFT JOIN `dw_vw.gam_impressions_rev_day` b ON gd.order_line_item_id = b.order_line_item_id
    AND gd.day_dt = b.day_dt
  GROUP BY 1,2,3
)

,delivered as (
  SELECT 
    fpa.billing_month
    ,fpa.sales_order_id
    ,fpa.sales_order_line_item_id
    ,fpa.sales_order_nm
    -- ,urc.product_nm
    ,fpa.sales_order_type_desc
    ,CASE
      WHEN fpa.billing_month < "2022-10-01"
        THEN "M&C"
      WHEN fpa.billing_month >= "2022-10-01" 
      AND fpa.brand IN(
        "GameFAQs" --GAMES
        ,"Gamespot"
        ,"Giantbomb"
        ,"Comicvine"
        ,"TV Guide" --ENTERTAINMENT 
        ,"Metacritic"
        ,"Metrolyrics"
        ,"TV.Com"
        ,"Chowhound"
        ,"Guide.com"
        ,"NoCable"
        ,"CordCutters")
        --OR fpa.product_nm LIKE "%RV Games%" -- or use urc's product_nm if not the same
      THEN "G&E"
      --WHEN fpa.product_nm LIKE "%Digital Lifestyle%"
      --THEN "M&C"
      ELSE "M&C"
    END AS company
    ,sum(fpa.recgnzed_rev_amount) recgnzed_rev_amount
    ,sum(fpa.recgnzed_rev_amount_usd) recgnzed_rev_amount_usd
    ,sum(fpa.invoice_amount) invoice_amount
    ,sum(fpa.invoice_amount_usd) invoice_amount_usd
  FROM rv-mt-data-prod.erp_vw.erp_fpa_report fpa
  WHERE billing_month between year_start and prev_month_end 
  GROUP BY 1,2,3,4,5,6
  ORDER BY sales_order_id, sales_order_line_item_id, billing_month
)

,booked_and_pg as(
  SELECT
    billing_month
    ,d.sales_order_id
    ,d.sales_order_line_item_id
    ,d.sales_order_nm
    ,d.sales_order_type_desc
    ,d.company
    ,mbr.line_start_dt
    ,mbr.line_end_dt
    , CASE
      WHEN EXTRACT(MONTH from mbr.line_end_dt) = EXTRACT(MONTH from d.billing_month)
      AND EXTRACT (YEAR from mbr.line_end_dt) = EXTRACT (YEAR from d.billing_month)
      THEN "Yes"
      ELSE "No"
      END AS line_ended_in_month
    ,mbr.op_booked_revenue_usd
    ,CASE
      WHEN billing_month < "2022-10-01"
        THEN "M&C"
      WHEN mbr.brand_nm IN (
        "GameFAQs" --GAMES
        ,"Gamespot"
        ,"Giantbomb"
        ,"Comicvine"
        ,"TV Guide" --ENTERTAINMENT 
        ,"Metacritic"
        ,"Metrolyrics"
        ,"TV.Com"
        ,"Chowhound"
        ,"Guide.com"
        ,"NoCable"
        ,"CordCutters")
      AND billing_month >= "2022-10-01"
      THEN "G&E"
      ELSE "M&C"  
      END AS booked_company
    ,CASE
      WHEN sum(ad_revenue) IS NOT NULL
        THEN sum(IFNULL(recgnzed_rev_amount_usd,0) + ad_revenue)
        ELSE sum(IFNULL(recgnzed_rev_amount_usd,0))
      END AS recgnzed_rev_amount_usd
    ,sum(invoice_amount_usd) as invoice_amount_usd
  FROM delivered d
  LEFT JOIN `rv-mt-data-prod.erp_vw.erp_monthly_booked_revenue` mbr
    on d.sales_order_id = mbr. sales_order_id
    and d.sales_order_line_item_id = mbr. sales_order_line_item_id
    and d.billing_month = mbr.booked_month
  LEFT JOIN pg_lines_by_month pg
    ON d.sales_order_line_item_id = pg.sfa_order_line_nbr
    AND d.billing_month = pg.delivered_month
  WHERE d.sales_order_type_desc IN ("Media","Intercompany","Custom Programmatic Guaranteed")
  AND mbr.sales_order_line_item_nm NOT LIKE "%The Points Guy%"
  AND mbr.sales_order_line_item_nm NOT LIKE "%TPG%"
  AND mbr.sales_order_line_item_nm NOT LIKE "%Lonely Planet%"
  AND mbr.sales_order_line_item_nm NOT LIKE "%Bankrate%"
  AND mbr.brand_nm NOT IN ("The Points Guy", 
    "Lonely Planet"
    ,"Accounting"
    ,"Accredited Schools Online"
    ,"Affordable Colleges Online"
    ,"Best College Reviews"
    ,"Best Colleges"
    ,"Best Counseling Degrees"
    ,"College Choice"
    ,"Computer Science"
    ,"Criminal Justice Degree Schools"
    ,"Cyber Degrees"
    ,"EDU"
    ,"Grad School Hub"
    ,"Higher Education-SAS"
    ,"Learn How To Become"
    ,"Nurse Journal"
    ,"Nurse Practitioner Schools"
    ,"Online MBA"
    ,"Psychology"
    ,"The Best Schools"
    ,"UNKNOWN")
  GROUP BY 1,2,3,4,5,6,7,8,9,10,11
  ORDER BY 2,3,1
)

,fpa_pg as(
  SELECT
    billing_month
    ,bpg.sales_order_id
    ,bpg.sales_order_line_item_id
    ,bpg.sales_order_nm
    ,bpg.sales_order_type_desc
    ,bpg.line_start_dt
    ,bpg.line_end_dt
    ,bpg.line_ended_in_month
    ,bpg.op_booked_revenue_usd
    ,bpg.booked_company
    ,bpg.company
  ,CASE
      WHEN bpg.company = "M&C"
        THEN sum(IFNULL(recgnzed_rev_amount_usd,0))
      END AS mc_revenue_usd
  ,CASE
      WHEN bpg.company = "G&E"
        THEN sum(IFNULL(recgnzed_rev_amount_usd,0))
      END AS ge_revenue_usd
    ,sum(bpg.invoice_amount_usd) invoice_amount_usd
  FROM booked_and_pg bpg
  WHERE op_booked_revenue_usd IS NOT null
AND sales_order_nm NOT LIKE "%Test%"
  -- and sales_order_id = 82331
  GROUP BY 1,2,3,4,5,6,7,8,9,10,11
  ORDER BY 2,3,1
)

,shifted_rev as(
  SELECT
    billing_month
    ,fpa.sales_order_id
    ,fpa.sales_order_line_item_id
    ,fpa.sales_order_nm
    ,fpa.sales_order_type_desc
    ,fpa.line_start_dt
    ,fpa.line_end_dt
    ,fpa.line_ended_in_month
    ,fpa.booked_company
    ,fpa.op_booked_revenue_usd
    ,CASE
      WHEN fpa.sales_order_type_desc = "Custom Programmatic Guaranteed"
        THEN sum(IFNULL(fpa.op_booked_revenue_usd,0) - sum(IFNULL(mc_revenue_usd,0) + IFNULL(ge_revenue_usd,0)))
          over (partition by fpa.sales_order_line_item_id order by fpa.billing_month rows between unbounded preceding and 1 preceding) 
        ELSE sum(IFNULL(fpa.op_booked_revenue_usd,0) - sum(IFNULL(mc_revenue_usd,0) + IFNULL(ge_revenue_usd,0)))
          over (partition by fpa.sales_order_line_item_id order by fpa.billing_month rows between unbounded preceding and 1 preceding) 
      END as shifted_revenue_usd
    ,sum(IFNULL(mc_revenue_usd,0)) mc_revenue_usd
    ,sum(IFNULL(ge_revenue_usd,0)) ge_revenue_usd
    ,sum(IFNULL(mc_revenue_usd,0)) + sum(IFNULL(ge_revenue_usd,0)) total_op_revenue_usd
    ,sum(invoice_amount_usd) invoice_amount_usd
  FROM fpa_pg fpa
  GROUP BY 1,2,3,4,5,6,7,8,9,10
  ORDER BY 2,3,1
)


SELECT
  billing_month
  
  ,sales_order_id
  ,sales_order_line_item_id
  ,sales_order_nm
  ,sales_order_type_desc
  ,line_start_dt
  ,line_end_dt
  ,line_ended_in_month
  ,booked_company
  ,op_booked_revenue_usd
  ,sum(shifted_revenue_usd) shifted_revenue_usd
  ,op_booked_revenue_usd + IFNULL(shifted_revenue_usd,0) as total_booked_revenue_usd
  ,sum(mc_revenue_usd) mc_revenue_usd
  ,sum(ge_revenue_usd) ge_revenue_usd
  ,sum(total_op_revenue_usd) total_op_revenue_usd
  ,sum(invoice_amount_usd) invoice_amount_usd
  ,sum(total_op_revenue_usd) - sum(invoice_amount_usd) recognized_vs_invoiced
  ,(IFNULL(mc_revenue_usd,0) + IFNULL(ge_revenue_usd,0)) - invoice_amount_usd as total_revenue_vs_invoice_usd
  ,(IFNULL(op_booked_revenue_usd,0) + IFNULL(shifted_revenue_usd,0)) - (IFNULL(mc_revenue_usd,0)+ IFNULL(ge_revenue_usd,0))  as missing_delivery_usd
  ,CASE
    WHEN line_ended_in_month = "Yes"
      AND (IFNULL(mc_revenue_usd,0) + IFNULL(ge_revenue_usd,0)) < (op_booked_revenue_usd + IFNULL(shifted_revenue_usd,0))
      THEN "UD"
    WHEN line_ended_in_month = "No"
      AND (IFNULL(mc_revenue_usd,0) + IFNULL(ge_revenue_usd,0)) < (op_booked_revenue_usd + IFNULL(shifted_revenue_usd,0))
      THEN "Late"
      ELSE "Early"
  END category
  ,EXTRACT(MONTH FROM billing_month) AS month_nbr
  #,EXTRACT(Year FROM billing_month) AS year_nbr
FROM shifted_rev
GROUP BY 1,2,3,4,5,6,7,8,9,10,12,18,19,20#,21
#HAVING sales_order_type_desc <> 'Custom Programmatic Guaranteed'
HAVING EXTRACT(Year FROM billing_month)= 2023 AND sales_order_id <> 80709
ORDER BY 2,3,1

""", project_id= project_id)

In [ ]:
#SFDC Opp Pipe BQ Data Pull

project_id = 'rv-mt-data-prod'

sfdc = pd.io.gbq.read_gbq("""SELECT opportunity_owner,
opportunity_nm,
opportunity_id_nbr,
SUM(opportunity_revenue_usd) AS schedule_amount_converted,

--CASE WHEN stage_nm IN ('Stage 5 - IO Received','Stage 6 - Campaign Live') THEN 'Booked'
--ELSE 
stage_nm As stage, --END AS stage,

probability_pct/100 AS probability_pct,
product_nm,
product_family,
 
CASE WHEN product_nm = 'Games - Mobile' THEN 'Games'
WHEN product_family ='Games' AND product_nm = 'RV Games - Mobile' Then 'RV Games'
WHEN product_family ='Rotational' AND product_nm = 'RV Games - Display' Then 'RV Games'
WHEN product_family ='Gamespot' Then 'Games'
WHEN product_family ='Games Network' THEN 'RV Games' 
WHEN product_family ='Games' AND product_nm = 'UK: Games Network' Then 'Games'
WHEN product_family ='Games' AND product_nm = 'AU: GameSpot.com' Then 'Games'
WHEN product_family ='Games' AND product_nm = 'RV Games - Video' Then 'RV Games'
WHEN product_family ='Games' AND product_nm LIKE('%RV%') Then 'RV Games'
WHEN product_family ='Rotational' AND product_nm LIKE('%RV%') Then 'RV Games'
WHEN product_family ='Games' AND product_nm = 'UK: GameSpot' Then 'Games'
WHEN product_family ='Games' AND product_nm ='US: Games Network' THEN 'Games'
WHEN product_family ='Games' AND product_nm ='UK: Games Network - USI' Then 'Games'
WHEN product_family = 'Rotational' AND product_nm LIKE('%Tech%') THEN 'Tech'
WHEN product_family = 'Rotational' AND product_nm LIKE('%Digital%') THEN 'Digital Lifestyle'
WHEN product_family = 'Rotational' AND product_nm LIKE('%UK%') THEN 'Tech'

#WHEN product_family = 'Rotational' AND site LIKE ('%Network|B2B%') THEN 'ZD Net'
#WHEN product_family = 'Rotational' AND site LIKE ('%Network|B2C%') THEN 'Entertainment'
##WHEN product_family = 'Rotational' AND site LIKE ('%Run of Tech%') THEN 'ZD Net'
#WHEN product_family = 'Rotational' AND site LIKE ('%Network|All%') THEN 'CNET'
#WHEN product_family = 'Rotational' AND site LIKE ('%Run of Cnet%') THEN 'CNET'
#WHEN product_family = 'Rotational' AND site LIKE ('%Run of CNET%') THEN 'CNET'
#WHEN product_family = 'Rotational' AND site LIKE ('%Rotational%') THEN 'CNET'
#WHEN product_family = 'Rotational' AND site LIKE ('%Tech - Games - Lifestyle%') THEN 'CNET'
#WHEN product_family = 'Games Network' Then 'Games'
WHEN product_family = 'B2B' AND site LIKE ('%CNET%') THEN 'CNET'
WHEN product_family = 'B2B' AND site LIKE ('%ZD%') THEN 'ZD Net'
WHEN product_family = 'B2B' AND site LIKE ('%B2B%') THEN 'ZD Net'
WHEN product_family = 'TV Guide' Then 'Entertainment'
WHEN product_family LIKE ('%TV%') Then 'Entertainment'
WHEN product_family = 'Tech & News' Then 'Tech'
WHEN product_family LIKE ('%CNET%') Then 'CNET'
WHEN product_family LIKE ('%Points%') Then 'RVMN'
WHEN product_family LIKE ('%Bank%') Then 'RVMN'
WHEN product_family LIKE ('%Lonely%') Then 'RVMN'
WHEN product_family LIKE ('%Credit%') Then 'RVMN'
WHEN product_family LIKE ('%Connect%') Then 'RVMN'
WHEN product_family LIKE ('%EDU%') Then 'RVMN'
WHEN product_family LIKE ('%College%') Then 'RVMN'
WHEN product_family LIKE ('%School%') Then 'RVMN'
ELSE product_family END AS product_family_rollup,
#schedule_date,
EXTRACT(ISOWEEK from schedule_date) AS week_nbr,
schedule_month,
EXTRACT(MONTH from schedule_month) AS month_nbr,
schedule_quarter,
EXTRACT(QUARTER from schedule_quarter) AS quarter_nbr,
fiscal_year,
opportunity_record_type,
include_in_probable_desc,
account_nm,
country,
--CASE WHEN product_family = 'Rotational' AND country = 'US' THEN 'US Rotl'
CASE WHEN country = 'US' THEN 'US'
WHEN product_family = 'Rotational' THEN 'US'
WHEN product_family LIKE ('%Games%') THEN 'US'
ELSE 'ROW' END AS country_group,
SUM(opportunity_revenue) AS schedule_amount,
operative_nbr As operative_order_nbr,
#"breakdown" AS B2B_breakdown_not_needed,
parent_account_nm As advertiser,
agency_nm,
"agency_parent"AS agency_parent_needed,
upfront_ind,
site,
rv_brand_nm,
current_date AS data_pulled,
schedule_date,
CASE WHEN opportunity_owner IN ('Adam Rabbani','Alex Hering','Aaron Brisco','Alexa Loebel',
'Daniel Riesenbach','Duncan Stanley','Katie Devine', 'Regan Kelly') THEN 'Patrick Shaughnessy'
WHEN opportunity_owner IN ('Alex Wolfe','Kristin Rachlin','Ryan Kahn','Tadeo Perez',
'Tricia Orth','Zach Verch', 'Darren McGee','Allison Nunez') THEN 'Eric Webber'
WHEN opportunity_owner IN ('Brian Chin','Daniel Dreyfus','Melissa Planty','Ravi Lakhani',
'Sam Talbot','Stephen Crawford','Alicia Weston') THEN 'Matt Rowell'
ELSE opportunity_owner END AS slt,

CASE WHEN product_nm LIKE('%Video%') THEN 'Video'
WHEN product_nm LIKE('%video%') THEN 'Video'
ELSE 'Display' END AS product_nm_rollup,
slt_leader_nm,


CASE WHEN account_category_nm LIKE('%Electronics%') THEN 'Tech'
WHEN account_category_nm LIKE('%B2B%') THEN 'Tech'
WHEN account_category_nm LIKE('%Incubator%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Media%') THEN 'M&E'
ELSE account_category_nm END AS category,




opportunity_category_nm AS opp_category,

CASE WHEN stage_nm IN ('Stage 5 - IO Received','Stage 6 - Campaign Live') THEN SUM(opportunity_revenue_usd) * 1.0
WHEN stage_nm IN ('Stage 4 - LOI / Verbal Commit') THEN SUM(opportunity_revenue_usd) * .75
WHEN stage_nm IN ('Stage 3 - Actively Engaged') THEN SUM(opportunity_revenue_usd) * .5
WHEN stage_nm IN ('Stage 2 - RFP Received') THEN SUM(opportunity_revenue_usd) * .25
WHEN stage_nm IN ('Stage 1 - Client Interaction') THEN SUM(opportunity_revenue_usd) * .10
WHEN stage_nm IN ('Stage 0 - Prospecting') THEN SUM(opportunity_revenue_usd) * .05
ELSE SUM(opportunity_revenue_usd) END AS weighted_schedule_amount_converted



 FROM `rv-mt-data-prod.dw_vw.sfdc_opportunity_pipeline`
 #WHERE CASE WHEN schedule_quarter >= '2022-01-01' THEN opportunity_owner NOT IN ('Patrick #Steele', 'Regan Kelly', 'Tanusha Sharma', 'Nixon Soh') END
 WHERE product_nm NOT IN('Programmatic - GameSpot Video','Asia: Direct: Basic Lead',
'Asia: Direct: Profiled Lead (1 to 4 questions)',
'AU: Direct: Basic Lead',
'AU: Direct: Direct Bypass',
'AU: Direct: Email Revenue',
'AU: Direct: Profiled Lead (1 to 4 questions)',
'AU: Direct: Profiled Lead (5 to 7 questions)',
'B2B Lead Gen',
'UK: Direct: Basic Lead',
'UK: Direct: Highly Qualified Lead',
'UK: Direct: Profiled Lead (1 to 4 questions)')

AND schedule_quarter BETWEEN '2023-01-01' AND '2023-10-01'



GROUP BY 1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,30,31,32,33,34,35
HAVING product_family_rollup IN ('CNET','Digital Lifestyle','RVMN','Tech','ZD Net')

""", project_id= project_id)

In [ ]:
#erp

project_id = 'rv-mt-data-prod'

erp_fpa = pd.io.gbq.read_gbq("""
SELECT sales_order_id,
sales_order_nm,
primary_salesperson_nm AS opportunity_owner,
sales_order_type_desc, 
CASE WHEN sales_order_type_desc LIKE ('%Open%') AND channel <> 'Programmatic PMP' THEN 'Open Exchange'
WHEN sales_order_type_desc LIKE ('%Open%') AND channel = 'Programmatic PMP' THEN 'PMP'
WHEN sales_order_type_desc LIKE ('%Guaranteed%') THEN 'Programmatic Guaranteed'
WHEN sales_order_type_desc = 'PMP' THEN 'PMP'
WHEN sales_order_type_desc IN ('Media')  THEN 'Direct Ad Sales'
WHEN sales_order_type_desc = 'Intercompany' THEN 'Direct Ad Sales'
ELSE 'N/A' END AS revenue_type,
sales_order_line_item_id,
gl_code,
company,
CASE WHEN company = 'RV025' THEN 'US'
WHEN company <> 'RV025' AND sales_order_type_desc LIKE ('%Guaranteed%') THEN 'ROW'
WHEN company = 'RV028' THEN 'EMEA'
WHEN company = 'RV026' THEN 'ANZ'
WHEN company = 'RV027' THEN 'APAC'
WHEN company = 'RV009' THEN 'US'
WHEN company = 'RV013' THEN 'Health Line'
ELSE company END AS region,
location,
channel_code,
channel,
revenue_category_code,
revenue_category,
business_unit,
CASE WHEN brand LIKE ('%Gamespot%') THEN 'Game Spot'
WHEN brand LIKE ('%Edu%') THEN 'EDU'
WHEN brand LIKE ('%Bank%') Then 'Bank Rate'
WHEN brand LIKE ('%Next%') THEN 'Next Advisor'
WHEN brand LIKE('%spot%') Then 'Game Spot'
WHEN brand LIKE ('%FAQs%') THEN 'Game FAQs'
WHEN brand LIKE ('%bomb%') THEN 'Giant Bomb'
WHEN brand LIKE ('%ZD%') THEN 'ZD Net'
WHEN brand LIKE ('%Comic%') THEN 'Comic Vine'
ELSE brand END AS brand,
customer_name AS advertiser,
billing_period_nm,
LEFT(billing_period_nm,3) AS month,
RIGHT(billing_period_nm,4)AS year,


SUM(recgnzed_rev_amount_usd)  AS erp_fpa_revenue,
SUM(invoice_amount_usd) AS erp_fpa_invoiced,
CASE WHEN LEFT(billing_period_nm,3) IN ("Jan","Feb","Mar") THEN 'Q1'
WHEN LEFT(billing_period_nm,3) IN ("Apr","May","Jun") THEN 'Q2'
WHEN LEFT(billing_period_nm,3) IN ("Jul","Aug","Sep") THEN 'Q3'
WHEN LEFT(billing_period_nm,3) IN ("Oct","Nov","Dec") THEN 'Q4'
ELSE billing_period_nm END AS quarter,


CASE WHEN LEFT(billing_period_nm,3) IN ("Jan") THEN '1'
WHEN LEFT(billing_period_nm,3) IN ("Feb") THEN '2'
WHEN LEFT(billing_period_nm,3) IN ("Mar") THEN '3'
WHEN LEFT(billing_period_nm,3) IN ("Apr") THEN '4'
WHEN LEFT(billing_period_nm,3) IN ("May") THEN '5'
WHEN LEFT(billing_period_nm,3) IN ("Jun") THEN '6'
WHEN LEFT(billing_period_nm,3) IN ("Jul") THEN '7'
WHEN LEFT(billing_period_nm,3) IN ("Aug") THEN '8'
WHEN LEFT(billing_period_nm,3) IN ("Sep") THEN '9'
WHEN LEFT(billing_period_nm,3) IN ("Oct") THEN '10'
WHEN LEFT(billing_period_nm,3) IN ("Nov") THEN '11'
WHEN LEFT(billing_period_nm,3) IN ("Dec") THEN '12'
ELSE billing_period_nm END AS month_nbr,
product_nm

 FROM `rv-mt-data-prod.erp_vw.erp_fpa_report` 
 WHERE billing_period_nm LIKE('%2023%')# OR billing_period_nm LIKE('%2021%')
AND company <>'RV013'
 AND brand NOT IN('Tech Republic', 'Guide.com')
 #AND channel <> 'Programmatic'
 GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25
 HAVING revenue_type <>'N/A' and revenue_type <>'Open Exchange'
AND sales_order_type_desc <>'Open Exchange-Programmatic'

""", project_id= project_id)

erp_fpa['month_nbr'] = erp_fpa['month_nbr'].astype(int)

In [ ]:
project_id = 'rv-mt-data-prod'

wkday =  pd.io.gbq.read_gbq("""
WITH wday AS(
SELECT accounting_dt AS day_dt,
EXTRACT(MONTH from accounting_dt) AS month_dt,
EXTRACT(QUARTER FROM accounting_dt) AS quarter_dt,
EXTRACT(YEAR from accounting_dt) AS year_dt,
company,


CASE WHEN LEFT(company,5) ='RV025' THEN 'US'
WHEN LEFT(company,5) ='RV028' THEN 'EMEA'
WHEN LEFT(company,5) ='RV026' THEN 'ANZ'
WHEN LEFT(company,5) ='RV027' THEN 'APAC'
WHEN LEFT(company,5) ='RV009' THEN 'US'
WHEN LEFT(company,5) ='RV013' THEN 'US'
WHEN LEFT(company,5) ='RV001' THEN 'Red Ventures'
WHEN LEFT(company,5) ='RV020' THEN 'Red Ventures'
WHEN LEFT(company,5) ='RV029' THEN 'Channel Community Network Corporation'
WHEN CONCAT(channel," _ ",revenue_category) IN("C501 Programmatic - Guaranteed _ Desktop","C501 Programmatic - Guaranteed _ Mobile Exchange","C501 Programmatic - Guaranteed _ Desktop Exchange","C501 Programmatic - Guaranteed _ Video","C501 Programmatic - Guaranteed _ Mobile","C501 Programmatic - Guaranteed") AND LEFT(company,5) ='RV026' THEN 'US'
WHEN LEFT(company,5) ='RV007' THEN 'Soda'
WHEN LEFT(company,5) ='RV022' THEN 'My Slumber Yard'
WHEN LEFT(company,5) ='RV009' THEN 'Cards (formerly Bankrate)'
WHEN LEFT(company,5) ='RV045' THEN 'Bankrate)'
ELSE LEFT(company,5) END AS region,

journal_number,
journal_name,
status AS ledger_status,
source_desc AS ledger_source,
ledger AS ledger_type,


operational_transaction,
line_memo,
CASE WHEN line_memo = 'Twitter Amplify May' THEN '81945'
WHEN line_memo LIKE ('%Twitter%') THEN '81760'
WHEN line_memo like('%reclass%') THEN 'Rev Reclass'
WHEN line_memo like('%Reclass%') THEN 'Rev Reclass'
WHEN line_memo like('%AU Enterprise%') THEN 'AU Enterprise'
WHEN line_memo like('%CCN: Google%') THEN 'CCN Google'
#WHEN line_memo like('CM|a||%') THEN 'No ID'
#WHEN line_memo like('%CM|%') THEN 'No ID'
WHEN line_memo like('%IBM IPC%') THEN 'IBM'
WHEN line_memo like('%IO#%') THEN SUBSTR(line_memo,5,5)
WHEN line_memo like('%NoCable:%') THEN 'No Cable'
ELSE LEFT(line_memo,5) END AS operative_order_id,
channel,
revenue_category,
CONCAT(channel," _ ",revenue_category) AS channel_revcat_cat,

CASE WHEN CONCAT(channel," _ ",revenue_category) IN (
"C010 Production Fees _ Other",
"C121 Video - Onsite _ Video",
"C265 Display _ Advertising -Digital",
#"C265 Display _ Advertising -Digital"
"C265 Display _ Desktop",
"C265 Display _ Mobile",
"C726 Email/Co-Reg _ Other",
"C957 Social Media _ Other",
"C957 Social Media _ Video"
) THEN 'Direct Ad Sales'

WHEN CONCAT(channel," _ ",revenue_category) IN(
  "C501 Programmatic - Guaranteed _ Desktop",
  #"C501 Programmatic - Guaranteed _ Mobile Exchange",
  #"C501 Programmatic - Guaranteed _ Desktop Exchange",
  "C501 Programmatic - Guaranteed _ Video",
  "C501 Programmatic - Guaranteed _ Mobile"
  #"C501 Programmatic - Guaranteed"
  ) THEN 'Programmatic Guaranteed'

WHEN CONCAT(channel," _ ",revenue_category) IN ("C002 Audio _ Desktop Exchange",
"C500 Programmatic _ Desktop Exchange",
"C500 Programmatic _ Desktop",
"C726 Email/Co-Reg _ Desktop Exchange",
"C500 Programmatic _ Mobile Exchange",
"C500 Programmatic _ Video Exchange",
"C500 Programmatic _ Video",
"C501 Programmatic - Guaranteed _ Mobile Exchange",
"C501 Programmatic - Guaranteed _ Desktop Exchange"
) THEN 'Open Exchange'

WHEN channel LIKE('%PMP%')# IN ("Programmatic PMP _ Desktop Exchange","Programmatic PMP _ #Mobile Exchange","Programmatic PMP _ Video Exchange") 
THEN 'PMP'

WHEN customer IN ('YouTube, LLC','Facebook Alpha Product','Google AdSense Service Agreement') THEN 'Other'
WHEN CONCAT(channel," _ ",revenue_category) IN ("C528 Paid Search") THEN 'Other'
WHEN business_unit IN ('B761 Giantbomb','B759 Gamefaqs','B774 Comic Vine','B760 Gamespot') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _ Subscriptions',
'C280 Other _ Merchandise',
'C280 Other _ Subscriptions',
'C120 Video – Offsite _ Subscriptions',
'C120 Video – Video',
'C120 Video – Video Exchange') THEN 'Other'
WHEN business_unit IN ('B762 TVGuide','B763 Metacritic','B765 TV.com','B766 Chowhound','B767 Guide.com','B771 Soda','B772 nocable','B773 CordCutters') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _ DAS','C120 Video – Offsite _ Video Exchange','C280 Other _ Licensing','C280 Other _ Other','C334 AllConnect.com _Advertising -Digital') THEN 'Other'

--Licensing Other
WHEN CONCAT(channel," _ ",revenue_category) IN ('C594 Partners _ Licensing',
'C594 Partners _ Activation / Installation', 'C280 Other _ Licensing','C280 Other _ Other') THEN 'Other'
WHEN business_unit LIKE('%CNET%') AND customer NOT LIKE ('%Faceb%') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _  Activation / Installation','C000 Admin _  Chargeback Expense (Contra Revenue)') THEN 'Other'
WHEN business_unit LIKE('%Download%') AND customer NOT LIKE ('%Faceb%') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _  Activation / Installation','C000 Admin _  Chargeback Expense (Contra Revenue)') THEN 'Other'

--transactional ecomm
WHEN CONCAT(channel," _ ",revenue_category) IN ('C594 Partners _ Leads',
'C594 Partners _ Leads - Exclusive','C594 Partners _ Other') THEN 'Commerce'


ELSE revenue_category END AS revenue_type_rollup,

business_unit,
CASE WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) = 'CMG' THEN 'CMG'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TPG%')  THEN 'The Points Guy'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Vine%') THEN 'Comic Vine'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) = 'Giantbomb' THEN 'Giant Bomb'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) = 'TIME' THEN 'Next Advisor'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%faqs%') THEN 'Game FAQs'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%spot%') THEN 'Game Spot'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Technology%') THEN 'CNET Tech'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Home%') THEN 'CNET Home'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Road%') THEN 'CNET Roadshow'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Exp%') THEN 'CNET Expansion'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%B2B - ZD%') THEN 'ZD Net'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%B2B - Tech%') THEN 'Tech Republic'
WHEN business_unit = 'B767 Guide.com' THEN 'Guide.com'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TVGuide%') THEN 'TV Guide'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Downl%') THEN 'Download'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Guide.com%') THEN 'Guide.com'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Chow%') THEN 'Chowhound'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Republic%') THEN 'Tech Republic'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Higher%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%ollege%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Education%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Learn%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Degree%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%school%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%grad%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Grad%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%General%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Soda%') THEN 'Soda'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%cable%') THEN 'No cable'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Cord%') THEN 'Cord Cutters'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Bank%') THEN 'Bank Rate'
ELSE RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) END AS brand,





CASE WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%CNET%') 
  AND LEFT(company,5) ='RV025' THEN 'CNET'
    WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%PriceBlink%') 
  AND LEFT(company,5) ='RV025' 
  AND CONCAT(channel," _ ",revenue_category) IN ('C594 Partners _ Leads')  THEN 'CNET'  
  
  WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Download%') 
  AND LEFT(company,5) ='RV025' THEN 'CNET'

     WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%CNET%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Down%')
  AND company NOT IN ("RV014 Healthline Holdings, Inc",
"RV015 Healthline UK",
"RV016 Healthline Iceland", 
"RV994 CNET Elimination", 
"RV999 Eliminations Company", 
"TR001 Tech Republic - US",
"TR002 Tech Republic - Australia",
"TR003 Tech Republic - Singapore",
"TR004 Tech Republic - UK",
"F001 Hudson Takeoff",
"RV025 CNET Media, Inc.") THEN 'CNET_ROW'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%ZD%') 
  AND LEFT(company,5) ='RV025' THEN 'B2B'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%ZD%') 
  AND LEFT(company,5) NOT IN ('F001','RV014','RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV025','RV024', 'RV042') THEN 'B2B_ROW'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Game%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Giant%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Comic%') 
  AND LEFT(company,5) NOT IN ('F001', 'RV014','RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV024', 'RV042') THEN 'Games'


WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%MySlumber%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Safety%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TIME%') 
  AND LEFT(company,5) NOT IN ('F001','RV014', 'RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV024', 'RV042') THEN 'Other'


WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TV%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Meta%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Metro%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Metro%')
OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Chow%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Soda%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%noCable%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Cord%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%nocable%')
OR business_unit = 'B767 Guide.com' 
AND LEFT(company,5) NOT IN ('F001','RV014', 'RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV024', 'RV042') THEN 'Entertainment'

ELSE "" END AS brand_region,

CASE WHEN business_unit IN ('B752 CNET Movers','B755 CNET Technology','B756 CNET Roadshow','B757 CNET Expansion','B770 Download.com') THEN 'CNET_Tech'
WHEN business_unit IN ('B830 CNET Media') THEN 'CNET_Media'
WHEN business_unit IN ('B828 CNET Broadband','B829 CNET Energy','B095 CNET Money') THEN 'CNET_Others'
WHEN business_unit = 'B930 Product' THEN 'Product'
WHEN business_unit = 'B199 MySlumberYard.com' THEN 'MSY'
WHEN business_unit = 'B404 Safety.com' THEN 'Safety'
WHEN business_unit = 'B753 B2B - ZDNet' THEN 'ZD_Net'
WHEN business_unit = 'B758 B2B - TechRepublic' THEN 'TR'
WHEN business_unit IN ('B759 Gamefaqs','B760 Gamespot','B761 Giantbomb','B774 Comic Vine','B762 TVGuide','B763 Metacritic','B773 CordCutters','B764 Metrolyrics','B754 M&T Entertainment','B751 Gaming') THEN 'G&E(Fandom)'
WHEN business_unit IN ('B771 Soda','B772 nocable','B765 TV.com','B766 Chowhound','B767 Guide.com') THEN 'G&E'
WHEN business_unit IN ('B548 TIME','B531 Next Advisor') THEN 'Time'
WHEN business_unit IN ('B350 CMG','B811 Media & Commerce Allocation','B768 CCS Data Services','B769 CCS Channel Online','B824 CNET Central Allocation','B831 CNET Group - Corporate','B010 Corporate') THEN 'Corporate'
WHEN business_unit IN ('B701 Imagitas-USPS','B822 MyMove Allocation','B700 Imagitas Corporate','B711 Imagitas-My Move','B712 Mymove.com','B713 Movers Club','B714 MyCOA') THEN 'Movers'
WHEN business_unit IN ('B040 Save on Energy','B041 Save on Energy - Commercial','B047 Choose Energy','B050 Solar Marketplace','B049 TXU Energy') THEN 'Energy'
WHEN business_unit = 'B450 Allconnect.com' THEN 'Broadband'
ELSE "" END AS updated_bu_map,



ledger_account,

CASE WHEN ledger_account IN ('5103:Product Cost', '5112:Other Cost of Sales', '5116:Direct Marketing', '5254:Freight', '5350:Sales and Use Tax') THEN 'Marketing Expenses'

WHEN ledger_account IN ('7000:Payroll Wages','7005:Payroll Taxes','7010:Bonuses','7011:Other Compensation','7025:Temporary Help', '7050:Benefits') THEN 'Head-Count Expenses'

WHEN ledger_account IN ('6010:Agent Payroll Taxes','6015:Agent Benefits','6030:Non-Agent Payroll Taxes', '6050:Other Agent Comp','6400:Facility Sales Center','7060:Recruiting','7100:Facility General & Administrative','7130:Company Functions','7135:Travel','7140:Meals & Entertainment','7142:Catering General & Administrative','7150:Employee Motivation','7160:Supplies',
'7170:Postage and Courier','7204:Licensing and Fees','7230:Other General & Administrative','7250:Research','7300:Technology','7800:Bad Debt','7999:Allocated Costs other','8200:Professional Services','8205:Dues and Subscriptions','8230:Public Relations','8240:General Insurance','8290:Contributions','8500:Café Management') THEN 'Non Head-Count Expense'

WHEN ledger_account = '8995:Vertical Expense Allocation' THEN 'Vertical Allocation Expense'
WHEN ledger_account = '8999:Shared Expense Allocation' THEN 'Corporate Allocation Expense'

WHEN ledger_account IN (
'4000:Revenue',
'4005:Other Revenue',
'4132:Chargeback Expense (Contra Revenue)')
#not originally toggled on
#"1102:Accrued Accounts Receivable")
THEN 'Revenue'
ELSE ledger_account END AS ledger_account_rollup,
cost_center,
customer,

SUM(translated_debit_minus_credit_amount) AS workday_revenue,
supplier_as_worktag,
spend_category_as_worktag,
spend_cat_id,
sales_order_id





FROM `rv-mt-data-prod.workday_vw.workday_journal` 

# Yr changed to 2023 due to data size and time to pull
WHERE accounting_dt >= '2023-01-01'
#AND operational_transaction LIKE('%Customer Invoice%')

--turn on later
#AND ledger_account IN ('4000:Revenue','4005:Other Revenue','4132:Chargeback Expense (Contra Revenue)',"1102:Accrued Accounts Receivable")

AND company NOT IN("RV014 Healthline Holdings, Inc",
"RV015 Healthline UK",
"RV016 Healthline Iceland", 
"RV994 CNET Elimination", 
"RV999 Eliminations Company", 
"TR001 Tech Republic - US",
"TR002 Tech Republic - Australia",
"TR003 Tech Republic - Singapore",
"TR004 Tech Republic - UK",
"RV021 Forward Home Lending, LLC",
"RV039 Lonely Planet Publications Inc.")

#AND line_memo Like '%IO#%'
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31
#HAVING revenue_type_rollup IN ('Direct Ad Sales', 'Programmatic Guaranteed') 

--turn on later
#AND ledger_account_rollup = 'Revenue'
),

sf AS (
SELECT 
CASE WHEN account_category_nm IN ('B2B', 'Consumer Electronics') THEN 'Tech'
WHEN account_category_nm LIKE('%Incubator%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Misc%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Social%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Beer%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Spirits%') THEN 'Incubator'
WHEN account_category_nm LIKE('%CPG%') THEN 'Incubator'
WHEN account_category_nm LIKE('%QSR%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Insurance%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Travel%') THEN 'Incubator'
ELSE account_category_nm END AS category,
operative_nbr,
opportunity_nm
#EXTRACT(MONTH from schedule_month) AS month_nbr,
#round(SUM(opportunity_revenue),2) AS sfdc_revenue
FROM `rv-mt-data-prod.dw_vw.sfdc_opportunity_pipeline` 
WHERE schedule_month >='2023-01-01'
AND probability_pct >= 90

AND product_nm NOT IN('Programmatic - GameSpot Video','Asia: Direct: Basic Lead',
'Asia: Direct: Profiled Lead (1 to 4 questions)',
'AU: Direct: Basic Lead',
'AU: Direct: Direct Bypass',
'AU: Direct: Email Revenue',
'AU: Direct: Profiled Lead (1 to 4 questions)',
'AU: Direct: Profiled Lead (5 to 7 questions)',
'B2B Lead Gen',
'UK: Direct: Basic Lead',
'UK: Direct: Highly Qualified Lead',
'UK: Direct: Profiled Lead (1 to 4 questions)')

AND opportunity_owner NOT IN('chinedu okadigbo', 'Craig Abley','Harry Michael','Hetvi Desai','Jackie McCarthy','Jahuie {Ja-Who-E} McCrary', 'Janet Tomokuni','Joe Mills','Kay Hope','Kevin Lichtman','Michael Macias','Nina Abuel','Nixon Soh','Nonsie Mtimkulu',
'Patrick Steele','Preye Tobolayefa','Roberto Sanha','Ross Arthurs','Ruby Anwar','Tanusha Sharma','Vivean Cheah')

GROUP BY 1,2,3
ORDER BY 1
)

SELECT 
wday.day_dt,
wday.month_dt AS month_nbr,
wday.quarter_dt AS quarter_nbr,
wday.year_dt,
wday.company,
wday.region,

wday.journal_number,
wday.journal_name,
wday.ledger_status,
wday.ledger_source,
wday.ledger_type,
wday.operational_transaction,
wday.line_memo,
#wday.operative_order_id AS wkday_order_id,
wday.sales_order_id AS wkday_order_id,
wday.channel_revcat_cat,

CASE #WHEN wday.brand_region = 'CNET' AND wday.channel_revcat_cat = 'C280 Other _ Licensing' THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET_ROW' AND wday.month_dt = 1 AND wday.year_dt = 2022 AND wday.channel_revcat_cat IN(TRIM('C280 Other _ Licensing')) THEN 'Other'
WHEN wday.brand_region = 'CNET_ROW' AND wday.channel_revcat_cat = 'C280 Other _ Licensing' THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET_ROW' AND wday.channel_revcat_cat = 'C000 Admin _ Other' THEN 'Other'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C265 Display _ Other' THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat ='C125 TIME Partnership _ Activation / Installation' THEN 'Commerce'
WHEN wday.brand_region = 'Entertainment' AND wday.channel_revcat_cat = 'C265 Display _ Advertising -Digital' THEN 'Direct Ad Sales'


WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C198 Natural Search _ Leads' THEN 'Commerce'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat ='C400 Affiliate _ Activation / Installation' THEN 'Commerce'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C407 MyFinance - RVMN _ Activation / Installation'  THEN 'Commerce'

WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C408 MyFinance - O&O - Bankrate _ Clicks' THEN 'Commerce'
#WHEN wday.brand_region = 'CNET' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET' AND wday.month_dt = 6 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'Other' AND wday.month_dt = 6 AND wday.channel_revcat_cat IN('C957 Social Media _ Desktop') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'B2B' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'Games' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'Entertainment' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'CNET_ROW' AND wday.month_dt = 3 AND wday.channel_revcat_cat IN('C280 Other _ Desktop') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'B2B' AND wday.month_dt = 3 AND wday.channel_revcat_cat IN('C726 Email/Co-Reg _ Leads') THEN 'Direct Ad Sales'
#WHEN wday.brand_region = 'CNET_ROW' AND wday.month_dt = 1 AND wday.channel_revcat_cat IN(TRIM('C280 Other _ Licensing')) THEN 'Other'
WHEN wday.brand_region = 'B2B_ROW' AND wday.month_dt = 1 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET' AND wday.month_dt = 7 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'
WHEN wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C120 Video - Offsite _ Video') THEN 'Direct Ad Sales'

WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C500 Programmatic _ Mobile') THEN 'Open Exchange'
WHEN wday.brand_region = 'B2B' AND wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C594 Partners _ Clicks') THEN 'Commerce'
WHEN wday.brand_region = 'Entertainment' AND wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C334 AllConnect.com _ Advertising -Digital') THEN 'Commerce'
WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 7 AND wday.channel_revcat_cat IN('C594 Partners _ Advertising -Digital','C594 Partners _ Activation / Installation','C594 Partners _ Chargeback Expense (Contra Revenue)') THEN 'Commerce'
WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt = 7 AND wday.channel_revcat_cat IN('C002 Audio _ Desktop Exchange') THEN 'Other'
WHEN wday.brand_region = 'Entertainment' AND wday.year_dt >= 2022 AND wday.month_dt = 7 AND wday.channel_revcat_cat IN('C500 Programmatic _ Other') THEN 'Open Exchange'
WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 7 AND wday.channel_revcat_cat IN('C500 Programmatic _ Mobile') THEN 'Open Exchange'

WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt = 8 AND wday.channel_revcat_cat IN('C002 Audio _ Desktop Exchange') THEN 'Other'
WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt >= 9 AND wday.channel_revcat_cat IN ('C002 Audio _ Other') THEN 'Open Exchange'
WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt >= 9 AND wday.channel_revcat_cat IN ('C002 Audio _ Desktop Exchange') THEN 'Open Exchange'
WHEN wday.brand_region = 'Entertainment' AND wday.year_dt >= 2022 AND wday.month_dt >= 9 AND wday.channel_revcat_cat IN ('C500 Programmatic _ Other') THEN 'Open Exchange'
WHEN wday.brand_region = 'CNET' AND wday.year_dt >= 2022 AND wday.month_dt >= 10 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'

WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 11 AND wday.channel_revcat_cat IN('C957 Social Media _ Desktop') THEN 'Direct Ad Sales'

WHEN wday.brand_region = 'CNET' AND wday.year_dt >= 2022 AND wday.month_dt >= 12 AND wday.channel_revcat_cat IN('C000 Admin _ Desktop') THEN 'Direct Ad Sales'


ELSE wday.revenue_type_rollup END AS revenue_type_rollup,
wday.business_unit,
wday.brand,

CASE WHEN wday.brand = 'PriceBlink' AND wday.ledger_account_rollup 
IN ('Non Head-Count Expense', "Head-Count Expenses") THEN 'CNET'
ELSE wday.brand_region END AS brand_region,
wday.ledger_account,
wday.ledger_account_rollup,
wday.customer,
wday.cost_center,

CASE WHEN wday.operative_order_id = '80709' THEN 'Tech'
WHEN wday.operative_order_id = '80137' THEN 'Tech'
WHEN wday.operative_order_id = '81000' THEN 'Finance'
WHEN wday.operative_order_id = '80223' THEN 'Consumer Electronics'
WHEN wday.operative_order_id = '05.20' THEN 'Finance'
WHEN wday.operative_order_id = 'VCBS' THEN 'M&E'
WHEN wday.operative_order_id = '82080' THEN 'Finance'
WHEN wday.operative_order_id = '80727' THEN 'M&E'
WHEN wday.operative_order_id = '80403' THEN 'M&E'
WHEN wday.operative_order_id = '81692' THEN 'Retail'
WHEN wday.operative_order_id = '82064' THEN 'M&E'
WHEN wday.operative_order_id = 'IBM' THEN 'Tech'
WHEN wday.line_memo = '07.2022 Cap One intercompany order' AND month_dt = 7 THEN 'Finance'
WHEN wday.operative_order_id = '81137' AND month_dt = 7 THEN 'Gaming'
WHEN wday.operative_order_id = '81136' AND month_dt = 7 THEN 'Gaming'
WHEN wday.line_memo = 'VCBS H1 FORFEIT Marketing credit' AND month_dt = 6 THEN 'M&E'
WHEN wday.operative_order_id = TRIM('Rev Reclass') AND month_dt = 6 THEN 'Finance'
WHEN wday.operative_order_id = TRIM('81481') AND month_dt = 6 THEN 'Retail'
WHEN wday.operative_order_id = TRIM('81547') AND month_dt = 6 THEN 'Retail'
WHEN wday.operative_order_id = TRIM('81574') AND month_dt = 6 THEN 'Retail'
WHEN wday.operative_order_id = TRIM('81426') THEN 'Retail'
WHEN wday.line_memo = '08.2022 Cap One intercompany order'  THEN 'Finance'
WHEN wday.operative_order_id = TRIM('81662') THEN 'Retail'
WHEN wday.operative_order_id = TRIM('Huawe') THEN 'Telco'
WHEN wday.line_memo LIKE ('%HUAWEI%') THEN 'Telco'
WHEN wday.operative_order_id = TRIM('80156') THEN 'Incubator'
WHEN wday.operative_order_id = TRIM('Comic') THEN 'Gaming'
WHEN wday.operative_order_id = TRIM('80333') THEN 'Telco'
WHEN wday.operative_order_id = TRIM('81327') THEN 'Incubator'
WHEN wday.operative_order_id = TRIM('82163') THEN 'Telco'
WHEN wday.line_memo = '09.2022 Cap One intercompany order'  THEN 'Finance'
WHEN wday.line_memo = 'VCBS Q3 FORFEIT Marketing credit'  THEN 'M&E'
WHEN wday.line_memo = '10.2022 Cap One intercompany order' THEN 'Finance'
WHEN wday.operative_order_id = TRIM('OPTUS') THEN 'Telco'

ELSE sf.category END AS category,
CASE WHEN wday.operative_order_id = '80709' THEN '47712_Google_AdX_RevShare_PG'	
ELSE sf.opportunity_nm END AS sales_order_nm,

SUM(wday.workday_revenue) AS workday_rev_actuals,
wday.supplier_as_worktag,
wday.spend_category_as_worktag,
wday.spend_cat_id,

CASE WHEN wday.brand IN( 'Entertainment','Games','Metacritic','TV.com') AND wday.year_dt >= 2022 AND wday.month_dt >= 10 THEN 'Fandom'
ELSE 'RV' END AS exclusion_filter,
updated_bu_map,

#CNET MEDIA
CASE WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ( 
  'C010 Production Fees _ Other',
'C121 Video - Onsite _ Video',
'C265 Display _ Desktop',
'C265 Display _ Mobile',
#'C280 Other _ Licensing',
'C501 Programmatic - Guaranteed _ Desktop',
'C501 Programmatic - Guaranteed _ Mobile',
'C501 Programmatic - Guaranteed _ Video',
'C501 Programmatic - Guaranteed _ Desktop Exchange',
'C501 Programmatic - Guaranteed _ Mobile Exchange',
'C501 Programmatic - Guaranteed _ Video Exchange',
'C265 Display _ Other',
'C265 Display _ Non-Direct') THEN 'DAS_Onsite_Rev'

WHEN  wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C265 Display _ Advertising -Digital') and customer LIKE('%Viacom%')THEN 'DAS_Onsite_Rev'

WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C957 Social Media _ Other') THEN 'DAS_Offsite_Rev(Social)'

WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN (
'C002 Audio _ Desktop Exchange',
'C500 Programmatic _ Desktop',
'C500 Programmatic _ Desktop Exchange',
'C500 Programmatic _ Mobile Exchange',
'C500 Programmatic _ Video Exchange',
'C500 Programmatic _ Video',
'C500 Programmatic _ Other',
'C528 Paid Search _ Clicks',
'C528 Paid Search _ Other',
'C726 Email/Co-Reg _ Desktop Exchange'
) THEN 'DAS_Onsite_Backfill_Rev(OE)'

WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('Programmatic PMP _ Desktop Exchange',
'Programmatic PMP _ Mobile Exchange','Programmatic PMP _ Video Exchange') THEN 'DAS_Onsite_Backfill_Rev(PMP)'



WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C120 Video - Offsite _ Video Exchange','C120 Video - Offsite _ Licensing') THEN 'DAS_Offsite_Backfill_Rev(YT)'
WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C594 Partners _ Licensing') THEN 'Licensing_Rev_Intl'
WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C280 Other _ Licensing') THEN 'Licensing_Rev_Brand'
WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C726 Email/Co-Reg _ Desktop Exchange','C726 Email/Co-Reg _ Other') THEN 'Email_Media_Rev'

#CNET Tech
WHEN wday.business_unit IN ('B752 CNET Movers','B755 CNET Technology','B756 CNET Roadshow','B757 CNET Expansion','B770 Download.com') AND wday.channel_revcat_cat ='C594 Partners _ Leads' THEN 'Transactional'

#CNET Others
WHEN wday.business_unit IN ('B828 CNET Broadband','B829 CNET Energy','B095 CNET Money') AND wday.channel_revcat_cat  IN('C470 CMG _ Clicks','C470 CMG _ Leads','C594 Partners _ Activation / Installation',
'C594 Partners _ Advertising -Digital',
'C594 Partners _ Chargeback Expense (Contra Revenue)', 
'C594 Partners _ Cart Compensation') THEN 'Transactional'

#ZD Net Original:
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C121 Video - Onsite _ Video','C265 Display _ Desktop','C265 Display _ Mobile','C726 Email/Co-Reg _ Other','C957 Social Media _ Other') THEN 'US_Direct_Ad_Sales(Discover)'
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C002 Audio _ Desktop Exchange','C500 Programmatic _ Desktop','C500 Programmatic _ Desktop Exchange','C500 Programmatic _ Mobile Exchange','C500 Programmatic _ Video Exchange',
#'C726 Email/Co-Reg _ Desktop Exchange', 'Programmatic PMP _ Desktop Exchange','Programmatic PMP _ Mobile Exchange','Programmatic PMP _ Video Exchange') THEN 'US_Programmatic(Discover)'
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C280 Other _ Licensing','C594 Partners _ Clicks','C594 Partners _ Desktop Exchange','C594 Partners _ Licensing') THEN 'US_Licensing(Discover)'
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C594 Partners _ Leads','C594 Partners _ Leads - Exclusive') THEN 'US_Commerce(Discover)'


WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ( 
  'C010 Production Fees _ Other',
'C121 Video - Onsite _ Video',
'C265 Display _ Desktop',
'C265 Display _ Mobile',
#'C280 Other _ Licensing',
'C501 Programmatic - Guaranteed _ Desktop',
'C501 Programmatic - Guaranteed _ Mobile',
'C501 Programmatic - Guaranteed _ Video',
'C501 Programmatic - Guaranteed _ Desktop Exchange',
'C501 Programmatic - Guaranteed _ Mobile Exchange',
'C501 Programmatic - Guaranteed _ Video Exchange',
'C265 Display _ Other',
'C265 Display _ Non-Direct') THEN 'DAS_Onsite_Rev'

WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C957 Social Media _ Other') THEN 'DAS_Offsite_Rev(Social)'

WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN (
'C002 Audio _ Desktop Exchange',
'C500 Programmatic _ Desktop',
'C500 Programmatic _ Desktop Exchange',
'C500 Programmatic _ Mobile Exchange',
'C500 Programmatic _ Video Exchange',
'C500 Programmatic _ Video',
'C500 Programmatic _ Other',
'C528 Paid Search _ Clicks',
'C528 Paid Search _ Other',
'C726 Email/Co-Reg _ Desktop Exchange'
) THEN 'DAS_Onsite_Backfill_Rev(OE)'

WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('Programmatic PMP _ Desktop Exchange',
'Programmatic PMP _ Mobile Exchange','Programmatic PMP _ Video Exchange') THEN 'DAS_Onsite_Backfill_Rev(PMP)'



WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C120 Video - Offsite _ Video Exchange','C120 Video - Offsite _ Licensing') THEN 'DAS_Offsite_Backfill_Rev(YT)'
WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C594 Partners _ Licensing') THEN 'Licensing_Rev_Intl'
WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C280 Other _ Licensing') THEN 'Licensing_Rev_Brand'
WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C726 Email/Co-Reg _ Desktop Exchange','C726 Email/Co-Reg _ Other') THEN 'Email_Media_Rev'









#Product
WHEN wday.business_unit IN ('B930 Product') AND wday.channel_revcat_cat IN('C594 Partners _ Leads') THEN 'Shopping'
WHEN wday.business_unit IN ('B930 Product') AND wday.channel_revcat_cat IN('C594 Partners _ Leads') THEN 'Coupons'

#MSY & Time
WHEN wday.business_unit IN ('B199 MySlumberYard.com','B404 Safety.com','B548 TIME') AND wday.channel_revcat_cat  IN('C500 Programmatic _ Desktop','C500 Programmatic _ Mobile') THEN 'Programmatic'
WHEN wday.business_unit IN ('B199 MySlumberYard.com','B404 Safety.com','B548 TIME') AND wday.channel_revcat_cat  IN('C410 MyFinance - Publisher Network _ Clicks') THEN 'Other'
WHEN wday.business_unit IN ('B199 MySlumberYard.com','B404 Safety.com','B548 TIME') AND wday.channel_revcat_cat  IN('C125 TIME Partnership _ Activation/Installation', 'C198 Natural Search _ Leads','C265 Display _ Desktop','C400 Affiliate _ Activation/Installation',
'C407 MyFinance - RVMN _ Activation/Installation','C408 MyFinance - O&O - Bankrate _ Clicks','C594 Partners _ Activation/Installation','C594 Partners _ Advertising - Digital','C594 Partners _ Chargeback Expense (Contra Revenue)') THEN 'Transactional'

ELSE updated_bu_map END AS updated_rev_type_rollup,

#New ledger rollups
CASE WHEN wday.ledger_account IN (
#'5116:Direct Marketing',
#'5350:Sales and Use Tax',
#'6010:Agent Payroll Taxes',
#'6015:Agent Benefits',
#'6400:Facility Sales Center',
#'7000:Payroll Wages',
#'7005:Payroll Taxes',
#'7010:Bonuses',
#'7011:Other Compensation',
#'7025:Temporary Help', 
#'7050:Benefits',


'5112:Other Cost of Sales',
'7060:Recruiting',
'7100:Facility General & Administrative',
'7130:Company Functions',
'7135:Travel',
'7140:Meals & Entertainment',
'7142:Catering General & Administrative',
'7150:Employee Motivation',
'7160:Supplies',
'7170:Postage and Courier',
'7204:Licensing and Fees',
'7230:Other General & Administrative',
'7250:Research',
'7300:Technology',
'7800:Bad Debt',
'7999:Allocated Costs other',
'8200:Professional Services',
'8205:Dues and Subscriptions',
'8230:Public Relations',
'8240:General Insurance',
'8290:Contributions',
'8500:Café Management'
)  #AND wday.business_unit NOT IN('B010 Corporate')
THEN 'm&c_hc_expenses'
ELSE ledger_account END AS expense_flag,

wday.business_unit AS business_unit_original


FROM wday
LEFT JOIN sf
ON wday.operative_order_id = sf.operative_nbr


GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33,34
#HAVING wday.revenue_type_rollup = 'Direct Ad Sales' OR wday.revenue_type_rollup = 'Programmatic Guaranteed'
HAVING ledger_account_rollup = 'Revenue' 
#DAS Only Filter, VCBS Q1 2023 FORFEIT Marketing credit  -->
AND updated_rev_type_rollup IN ('DAS_Offsite_Rev(Social)','DAS_Onsite_Rev','Email_Media_Rev') AND brand IN ('CNET Media','ZD Net')
#:Accrued Accounts Receivable'
ORDER BY year_dt DESC


""", project_id=project_id)
wkday['sales_order_nm']= wkday['sales_order_nm'].fillna("blank")
wkday['supplier_as_worktag']= wkday['supplier_as_worktag'].fillna("blank")
wkday['spend_category_as_worktag']= wkday['spend_category_as_worktag'].fillna("blank")
wkday['wkday_order_id']= wkday['wkday_order_id'].fillna(12345)


# Mth/Qtr Filters, Defs,DF changes
> BY Month (A.J.)
>> Defs, Month Filter, Data Updates, Stage Splits

In [ ]:
def qtr_filter(df,schedule_qtr):
    '''dynmamic date filter.  Will return data for quarter selection'''
    #target_mth = input().astype(int)
    print ('dataframe will be filtered and populate quarter:',schedule_qtr)
    df = df[(df['quarter_nbr'].eq(schedule_qtr))]
   
    return df


def mth_filter(df,schedule_mth):
    '''dynmamic date filter.  Will return data for month selection'''
    #target_mth = input().astype(int)
    print ('dataframe will be filtered and populate month:',schedule_mth)
    df = df[(df['month_nbr'].eq(schedule_mth))]
   
    return df

def mth_fwd_filter(df,schedule_mth):
    '''dynmamic date filter.  Will return data for month selection and following month.  Used to verify accounting orders that dont line up to erp_fpa IN MONTH'''
    #target_mth = input().astype(int)
    print ('dataframe will be filtered and populate month:',schedule_mth)
    #or filters
    df = df[(df['month_nbr'].eq(schedule_mth)) |  (df['month_nbr'].eq(schedule_mth+1))]
    #df = df[(df['month_nbr'].eq(schedule_mth+1))]
   
    return df    


    

In [ ]:
# SFDC changing 90 and 100 to "booked"

sfdc.loc[sfdc['stage'].eq('Stage 5 - IO Received'), 'stage'] = 'Booked'
sfdc.loc[sfdc['stage'].eq('Stage 6 - Campaign Live'), 'stage'] = 'Booked'
sfdc.loc[sfdc['category'].isnull(),'category'] = 'categorization_n/a'

# SFDC categorization n/a

sfdc['stage'] = sfdc['stage'].str.replace(' ', '_')
sfdc['stage'] = sfdc['stage'].str.replace('-', '_')
sfdc['stage'] = sfdc['stage'].str.replace('/', '_')
#sfdc['product_family_rollup'] = sfdc['product_family_rollup'].str.replace(' ', '_')

In [ ]:
#@title
#DF Month Filter

schedule_mth = 4
fwd_mth = schedule_mth +1

schedule_qtr = 1

sfdc= mth_filter(sfdc,schedule_mth)
op1 = mth_filter(op1,schedule_mth)
erp_fpa = mth_filter(erp_fpa,schedule_mth)
op_fi = mth_filter(op_fi,schedule_mth)


#Copying workday table to verify Ledger Info across multiple months
wkday_date = wkday

#Select with wkday filter to toggle on.  Current Month or Current and Following Month.  
wkday = mth_filter(wkday,schedule_mth)
#wkday = mth_fwd_filter(wkday,schedule_mth)
#wkday = wkday.fillna("blank")





dataframe will be filtered and populate month: 4
dataframe will be filtered and populate month: 4
dataframe will be filtered and populate month: 4
dataframe will be filtered and populate month: 4
dataframe will be filtered and populate month: 4


In [ ]:
wkday.month_nbr.value_counts()

4    411
Name: month_nbr, dtype: Int64

In [ ]:
#pd.set_option('display.max_columns', None)
#sfdc[sfdc['opportunity_nm']== 'IPC IBM 2023 DEFERRAL']#.weighted_schedule_amount_converted.sum()
# Give None SalesForce Op Order numbers Numberical value

sfdc.loc[(sfdc['operative_order_nbr'].isnull() ),'operative_order_nbr'] = str(12345)
sfdc.loc[(sfdc['advertiser'].isnull() ),'advertiser'] = 'Advertiser N/A'


#op1.loc[(op1['sales_order_id'].isnull() ),'sales_order_id'] = 12345
#op1.loc[(op1['advertiser_nm'].isnull() ),'advertiser_nm'] = 'Advertiser N/A'


In [ ]:
#sfdc[sfdc['opportunity_nm']== 'IPC IBM 2023 DEFERRAL'].info()

In [ ]:
stages = sfdc.stage.unique()
stages

array(['Booked', 'Stage_1___Client_Interaction', 'Stage_2___RFP_Received'],
      dtype=object)

In [ ]:
stages = sfdc.stage.unique()
families = sfdc.product_family_rollup.unique()

sfdc_rev_rollups = ['weighted_schedule_amount_converted']



def stage_split(stage):
    '''creates a seperate df for each brand_region '''
    df = sfdc[(sfdc['stage'] == stage)]
    
    return df     

for stage in stages:
    '''to call a vertical...stage_nm'''
    globals()[str(stage)] = stage_split(stage)

"""_________________________________________________"""

def family_split(family):
    '''creates a seperate df for each brand_region '''
    df = sfdc[(sfdc['product_family_rollup'] == family)]
    
    return df     

for family in families:
    '''to call a family...Digital_Lifestyle'''
    globals()[str(family)] = family_split(family)

sfdc_booked = Booked    

In [ ]:
#workday filter and pivot

#wkday pivot
#columns to filter on[journal_name, line_memo,'wkday_order_id,'brand','customer'sales_order_nm,supplier_as_worktag,spend_category_as_worktag, 'workday_rev_actuals]

def df_filter(df,*args):

  'df filter where we can filter for specific multiple column names, ex.) Jan or BU'
  df= df[[*args]]
  return df

def convert(lst):
    lst = np.array(lst)
    return list(-lst)  





def wkday_deets(df,order_id,*args):
  ''' filters dataframe by order ID, and groups, sums against columns chosen'''
 
    
  df = df[df['wkday_order_id']== order_id]
  #df = df.groupby([*args])['workday_rev_actuals'].sum().reset_index()
  df = pd.pivot_table(df, index = [*args],  values = 'workday_rev_actuals', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()#[:-1]
  lst = df['workday_rev_actuals'].astype(float)
  trans = pd.DataFrame(convert(lst),columns = ['workday_rev_actuals'])
  trans.rename(columns = {'workday_rev_actuals':'wkday_rev'},inplace = True)
  df = pd.concat([df,trans], axis = 1)
  df = df.drop(columns = ['workday_rev_actuals'])


  return df

def wkday_deets_piv(df,order_id,*args):
  ''' filters dataframe by order ID, and groups, sums against columns chosen then pivots across acct dates'''
 
    
  df = df[df['wkday_order_id']== order_id].reset_index()
  
  lst = df['workday_rev_actuals'].astype(float)
  trans = pd.DataFrame(convert(lst),columns = ['workday_rev_actuals'])
  trans.rename(columns = {'workday_rev_actuals':'wkday_rev'},inplace = True)
  df = pd.merge(df, trans, left_index = True, right_index = True)

  df = pd.pivot_table(df, index = [*args], columns = 'day_dt',  values = 'wkday_rev', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()#[:-1]


 

  return df

#____________________________________________________________________________________________________________________________________________________________________________________________________________


def erp_deets(df,order_id,*args):
  df = df[df['sales_order_id']== order_id] 
  df = pd.pivot_table(df, index = [*args],  values = ['erp_fpa_invoiced','erp_fpa_revenue'], aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()#[:-1]
  return df


# Pivot Merge and Join

In [ ]:
#@title
#Total sum per row: 
#G&E Orders should no longer be present.  

op1_piv = pd.pivot_table(op1, index = ['sales_order_nm','sales_order_id'],  values = 'Revenue', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
op1_piv.rename(columns = {'Revenue':'op1_revenue'},inplace = True)

sfdc_piv = pd.pivot_table(sfdc_booked, index = ['opportunity_nm','operative_order_nbr','advertiser'],  values = 'schedule_amount_converted', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
sfdc_piv.rename(columns = {'schedule_amount_converted':'sfdc_revenue'},inplace = True)
sfdc_piv.rename(columns = {'operative_order_nbr':'sales_order_id'},inplace = True)
sfdc_piv.rename(columns = {'opportunity_nm':'sales_order_nm'},inplace = True)

erp_rec = pd.pivot_table(erp_fpa, index = ['sales_order_id','sales_order_nm','advertiser'],values = 'erp_fpa_revenue', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
erp_rec['erp_fpa_revenue'] = erp_rec['erp_fpa_revenue'].fillna(0)

pd.set_option('display.max_rows', None)

erp_inv = pd.pivot_table(erp_fpa, index = ['sales_order_nm','advertiser'],values = 'erp_fpa_invoiced', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
erp_inv['erp_fpa_invoiced'] = erp_inv['erp_fpa_invoiced'].fillna(0)

#op Fi booked+shifts, we will plug into variance report
op_fi_piv = pd.pivot_table(op_fi, index = ['sales_order_id','sales_order_nm'],values = 'total_booked_revenue_usd', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
op_fi_piv.rename(columns = {'total_booked_revenue_usd':'op1_shifted_rev'},inplace = True)

op_booked_fi_piv = pd.pivot_table(op_fi, index = ['sales_order_id','sales_order_nm'],values = 'op_booked_revenue_usd', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
op_booked_fi_piv.rename(columns = {'op_booked_revenue_usd':'op1_fi_booked_rev'},inplace = True)



wkday_piv = pd.pivot_table(wkday, index = ['sales_order_nm','wkday_order_id','customer'],  values = 'workday_rev_actuals', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
lst = wkday_piv['workday_rev_actuals'].astype(float)
trans = pd.DataFrame (convert(lst),columns = ['wkday_revenue'])                     
wkday_piv = pd.concat([wkday_piv, trans],axis = 1)
wkday_piv = wkday_piv.drop(columns = ['workday_rev_actuals'])


sf_op1 =pd.merge(sfdc_piv, op1_piv, on = ['sales_order_nm'], how = 'outer', indicator = True)
sf_op1['revenue_delta'] = sf_op1['sfdc_revenue'] - sf_op1['op1_revenue']
#sf_op1['revenue_delta_%'] = (sf_op1['sfdc_revenue'] - sf_op1['op1_revenue'])/sf_op1['sfdc_revenue'].round(2)
pd.set_option('display.float_format', '${:,.0f}'.format)

order = ['advertiser', 'sales_order_nm','sales_order_id_x','sfdc_revenue','sales_order_id_y','op1_revenue','_merge','revenue_delta']

sf_op1 = sf_op1.reindex(columns = order).sort_values(by = ['sfdc_revenue'],ascending = False)
sf_op1.loc['Total'] = sf_op1.sum(numeric_only=True)

#sf_op1
#left = sfdc
#right =Op1 

# Revenue System Comparison

In [ ]:
#@title

print("SFDC_booked_rev:",sfdc_piv.sfdc_revenue.sum().astype(int),"\n")
print("Op1_booked_rev:",op1_piv.op1_revenue.sum().astype(int),"\n"),
print("ERP_FPA delivered/rec_rev:",erp_rec.erp_fpa_revenue.sum().astype(int),"\n"),
print("ERP_FPA invoiced_rev:",erp_inv.erp_fpa_invoiced.sum().astype(int),"\n"),
print("WKday_p&l_rev:",wkday_piv.wkday_revenue.sum().astype(int),"\n")

print("Difference between Recognized and Invoiced ERP",erp_rec.erp_fpa_revenue.sum().astype(int) - erp_inv.erp_fpa_invoiced.sum().astype(int),"\n")

#print("Difference between Delivered and P&L",erp_rec.erp_fpa_revenue.sum().astype(int) - wkday_piv.wkday_revenue.sum().astype(int))
print("Difference between Delivered and P&L",wkday_piv.wkday_revenue.sum().astype(int) - erp_rec.erp_fpa_revenue.sum().astype(int) )




SFDC_booked_rev: 1004294 

Op1_booked_rev: 913585 

ERP_FPA delivered/rec_rev: 865990 

ERP_FPA invoiced_rev: 858429 

WKday_p&l_rev: 850976 

Difference between Recognized and Invoiced ERP 7561 

Difference between Delivered and P&L -15014


In [ ]:
#@title
#pd.merge(sfdc_piv, op1_piv, on = ['sales_order_nm'], how = 'outer', indicator = True)

#all = pd.merge(pd.merge(sfdc_piv, op1_piv, on = ['sales_order_nm'], how = 'outer',indicator = True),erp,on = ['sales_order_nm'], how = 'outer', indicator = 'exists')
#all = pd.merge(pd.merge(pd.merge(sfdc_piv, op1_piv, on = ['sales_order_nm'], how = 'outer',indicator = True),erp_rec,on = ['sales_order_nm'], how = 'outer', indicator = 'exists'),wkday_piv, on =  ['sales_order_nm'], how = 'left')
all = pd.merge(pd.merge(pd.merge(pd.merge(sfdc_piv, op1_piv, on = ['sales_order_nm'], how = 'outer',indicator = True),
                                 erp_rec,on = ['sales_order_nm'], how = 'outer', indicator = 'exists'),
                                 erp_inv,on = ['sales_order_nm'], how = 'outer'),
                                 wkday_piv, on =  ['sales_order_nm'], how = 'outer')

all.rename(columns ={'sales_order_id_x':'sfdc_order_id','advertiser_x':'sfdc(booked)_advertiser','sales_order_id_y':'op1_order_id','advertiser_y':'erp(delivered)_advertiser','sales_order_id':'erp_order_id','customer':'wkday(invoice)_advertiser'},inplace = True)
#order = ['sales_order_nm','sfdc_order_id','op1_order_id','erp_order_id','wkday_order_id','sfdc(booked)_advertiser','erp(delivered)_advertiser', 'wkday(invoice)_advertiser','sfdc_revenue','op1_revenue','erp_fpa_revenue','erp_fpa_invoiced','wkday_revenue']
#all = all.loc[:,~all.columns.duplicated()]
#all = all.reindex(columns = order).sort_values(by = ['wkday_revenue'], ascending = False)#.reset_index()

#add in Op_fi shifted revenue, added late
all = pd.merge(all, op_fi_piv, on = ['sales_order_nm'], how = 'outer')

all = pd.merge(all, op_booked_fi_piv, on = ['sales_order_nm'], how = 'outer')

order = ['sales_order_nm','sfdc_order_id','op1_order_id','erp_order_id','wkday_order_id','sfdc(booked)_advertiser','erp(delivered)_advertiser', 'wkday(invoice)_advertiser','sfdc_revenue',
         'op1_revenue','op1_fi_booked_rev','op1_shifted_rev','erp_fpa_revenue','erp_fpa_invoiced','wkday_revenue']
#all = all.loc[:,~all.columns.duplicated()]
all = all.reindex(columns = order).sort_values(by = ['wkday_revenue'], ascending = False)#.reset_index()

all = all.fillna(0)

# shows the differences between what hits p&l vs what is booked, 
#positive value means P&L earning more than potential delivery
#negative value means P&L earning less than what order delivered, 
all['p&l:delivery'] =   all['wkday_revenue'] - all['erp_fpa_revenue'] 

all.loc['Total'] = all.sum(numeric_only=True)

#all[all.index.duplicated()]

all


,sales_order_nm,sfdc_order_id,op1_order_id,erp_order_id,wkday_order_id,sfdc(booked)_advertiser,erp(delivered)_advertiser,wkday(invoice)_advertiser,sfdc_revenue,op1_revenue,op1_fi_booked_rev,op1_shifted_rev,erp_fpa_revenue,erp_fpa_invoiced,wkday_revenue,p&l:delivery
29,ViacomCBS_Premieres_Q22023,82600,"$82,600","$82,600","$82,600",VIACOMCBS,ViacomCBS,ViacomCBS- Paramount,"$150,000","$150,000","$150,000","$150,000","$150,000","$150,000","$150,000",$0
4,Boost Mobile - SIM Media Strategy (Articles/So...,82239,"$82,239","$82,239","$82,239",DISH NETWORK,Boost Mobile,Boost Mobile,"$123,080","$123,080","$123,081","$-3,897","$106,491","$106,491","$106,491",$0
32,47712_Google_AdX_RevShare_PG,0,$0,"$80,709","$80,709",0,Google SF,Google AdX_RevShare,$0,$0,$0,$0,"$85,728","$85,728","$85,683",$-45
12,HPE_FY23_Annual Unified Campaign,82452,"$82,452","$82,452","$82,452",HEWLETT-PACKARD ENTERPRISE,Hewlett-Packard Enterprise,Hewlett-Packard Enterprise,"$68,725","$68,727","$68,728","$69,397","$69,381","$69,381","$69,128",$-252
24,Samsung_CTV Premium_Q2'23,82585,"$82,585","$82,585","$82,585",SAMSUNG ELECTRONICS CO LTD,Samsung Electronics North America,Starcom - Samsung,"$49,091","$49,091","$49,091","$49,091","$53,621","$53,621","$53,622",$0
9,Cisco Thought Leadership FY23 1H,82554,"$82,554","$82,554","$82,554",CISCO SYSTEMS INC,Cisco Systems Inc,Golin,"$28,750","$28,750","$28,750","$50,000","$50,000","$50,000","$50,000",$0
25,TMobile_HINT_Q2,82595,"$82,595","$82,595","$82,595",DEUTSCHE TELEKOM AG,T-Mobile,Initiative Media - T-Mobile,"$47,500","$47,500","$47,499","$47,499","$47,500","$47,500","$47,500",$0
18,Nutanix_FY23Q3_Global Brand AMS_ZDNet,82558,"$82,558","$82,558","$82,558",NUTANIX INC.,Nutanix Inc.,"Media Matters SF, LLC","$19,672","$19,672","$19,672","$19,672","$31,078","$31,078","$31,079",$0
8,Cisco Meraki_CNET & ZDNet Partnership 2023,82552,"$82,552","$82,552","$82,552",CISCO SYSTEMS INC,Cisco Systems Inc,NP Digital,"$29,420","$29,420","$29,420","$27,614","$27,614","$27,614","$27,615",$0
31,Wayfair - WayDay (April),82611,"$82,611","$82,611","$82,611",WAYFAIR,Wayfair.com,Wayfair.com,"$25,000","$25,000","$25,000","$25,000","$25,000","$25,000","$25,000",$0


# P&L versus Delivery 

In [ ]:
#@title

pd.set_option('display.float_format', '${:,f}'.format)
vz = all[['sales_order_nm','wkday_order_id','wkday(invoice)_advertiser',	'p&l:delivery']].sort_values('p&l:delivery', ascending = False)

vz = vz.loc[(vz['p&l:delivery'] >1) | (vz['p&l:delivery'] < -1) ].drop(['Total']) 
vz['wkday_order_id'] = vz['wkday_order_id'].astype(object)
vz.loc['Total'] = vz.sum(numeric_only=True)
pd.set_option('display.float_format', '${:,.0f}'.format)

vz

,sales_order_nm,wkday_order_id,wkday(invoice)_advertiser,p&l:delivery
41,Intel_Arc_Affiliate_2023,"$82,593",INTEL CORP,"$17,300"
40,Instacart SMB,"$82,580",Instacart Company,"$13,955"
47,blank,"$80,156",Facebook Alpha Product,"$4,144"
34,BBUY_COMP_Q12023_FY24 Q1 Intel Refresh,"$82,564",Starcom C/O: Re:Sources,$641
48,blank,"$81,329",SnapChat - CNET,$479
42,Lenovo_ZDNet_AUS_Q1'23_AUD,"$82,581",Scroll Media Limited,$49
36,C1_TECH_23_Q1_BM_TNAT_D_Tech National_TCHM_NA,"$82,494",Horizon Media/Altice,$6
49,tmo_postpaid_voice_x_pro_priority_ad18to54_x_e...,"$82,516",Initiative Media - Amazon,$-3
32,47712_Google_AdX_RevShare_PG,"$80,709",Google AdX_RevShare,$-45
6,CS_Algolia_Q1_FY24_B2C Ecommerce_RedVentures,"$82,578",Realm B2B Agency,$-189


# DF Details Selection
> Workday Details,
> erp_fpa Details

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

#wkday,'82580','day_dt','wkday_order_id','sales_order_nm','customer','line_memo','brand','updated_rev_type_rollup','channel_revcat_cat','business_unit'
#ledger_blanks = 12345

wkday_deets(wkday,12345,'day_dt','wkday_order_id','sales_order_nm','customer','updated_rev_type_rollup','channel_revcat_cat','line_memo')

,day_dt,wkday_order_id,sales_order_nm,customer,updated_rev_type_rollup,channel_revcat_cat,line_memo,wkday_rev
0,2023-04-30,12345,blank,CNET - Mymove,DAS_Onsite_Rev,C265 Display _ Non-Direct,"Mover Access Email Campaign, Timing: April 2023","$-2,560"
1,2023-04-30,12345,blank,GroupM,DAS_Onsite_Rev,C265 Display _ Desktop,IO 82471 - APR PROVISION 14%,"$-1,630"
2,Total,,,,,,,"$-4,190"


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

#wkday,'82580','day_dt','wkday_order_id','sales_order_nm','customer','line_memo','brand','updated_rev_type_rollup','channel_revcat_cat','business_unit'
#ledger_blanks = 12345

wkday_deets_piv(wkday_date,12345,'wkday_order_id','sales_order_nm','customer','brand', 'updated_rev_type_rollup','channel_revcat_cat','line_memo')#.info()

day_dt,wkday_order_id,sales_order_nm,customer,brand,updated_rev_type_rollup,channel_revcat_cat,line_memo,2023-01-31,2023-02-28,2023-03-01,2023-03-31,2023-04-30,Total
0,12345,blank,April Six/AMD,CNET Media,DAS_Onsite_Rev,C265 Display _ Desktop,AMD Revenue Correction December 2022 in January,"$140,868",$0,0,$0,$0,"$140,868"
1,12345,blank,April Six/AMD,ZD Net,DAS_Onsite_Rev,C265 Display _ Desktop,AMD Revenue Correction December 2022 in January,"$140,868",$0,0,$0,$0,"$140,868"
2,12345,blank,Boost Mobile,CNET Media,DAS_Onsite_Rev,C265 Display _ Desktop,Reclass - IO# 82239 - Invoice 206386 - Accrual December-2022,"$-150,000",$0,0,$0,$0,"$-150,000"
3,12345,blank,CNET - Mymove,CNET Media,DAS_Onsite_Rev,C265 Display _ Non-Direct,Correction of JE RV025-JE_00003617,$0,$0,0,"$-8,010",$0,"$-8,010"
4,12345,blank,CNET - Mymove,CNET Media,DAS_Onsite_Rev,C265 Display _ Non-Direct,"Mover Access Email Campaign, Timing: April 2023",$0,$0,0,$0,"$-2,560","$-2,560"
5,12345,blank,CNET- Affiliates,ZD Net,DAS_Onsite_Rev,C265 Display _ Other,Sponsored Post,$0,$0,10000,$0,$0,"$10,000"
6,12345,blank,GroupM,CNET Media,DAS_Onsite_Rev,C265 Display _ Desktop,IO 82471 - APR PROVISION 14%,$0,$0,0,$0,$-810,$-810
7,12345,blank,GroupM,CNET Media,DAS_Onsite_Rev,C265 Display _ Desktop,IO 82471 - QTR 1 PROVISION 14%,$0,$0,0,"$-41,426",$0,"$-41,426"
8,12345,blank,GroupM,ZD Net,DAS_Onsite_Rev,C265 Display _ Desktop,IO 82471 - APR PROVISION 14%,$0,$0,0,$0,$-820,$-820
9,12345,blank,GroupM,ZD Net,DAS_Onsite_Rev,C265 Display _ Desktop,IO 82471 - QTR 1 PROVISION 14%,$0,$0,0,"$-6,692",$0,"$-6,692"


In [ ]:
erp_deets(erp_fpa,82593,'sales_order_nm', 'advertiser','sales_order_line_item_id', 'product_nm')

,sales_order_nm,advertiser,sales_order_line_item_id,product_nm,erp_fpa_invoiced,erp_fpa_revenue
0,Total,,,,0,0


In [ ]:
#@title

#Fi booked rev by categories
#used to determine difference between Op1 Booked and Delivered
op_rev_cats = pd.pivot_table(op_fi, index = ['billing_month','category'],  values = 'missing_delivery_usd', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index()[:-1]
op_rev_cats = op_rev_cats.set_index('category')
op_rev_cats


# add to month for early delivery, so we flip value positive
early = -(op_rev_cats.loc['Early'].missing_delivery_usd.sum())


ud = -(op_rev_cats.loc['UD'].missing_delivery_usd.sum())

# late requires incorporation of PG delivered vs booked
late = op_rev_cats.loc['Late'].missing_delivery_usd.sum()

#PG Differences to add to late category 80709 Google SF*
pg_delivered =  erp_rec[erp_rec['sales_order_id']== 80709].erp_fpa_revenue.sum()#.astype(int)
pg_delivered

#Booked PG Orders, shifted taken into account, can bring overall booked number up or down
pg_shifts = op_fi[op_fi['sales_order_type_desc']=='Custom Programmatic Guaranteed']['mc_revenue_usd'].sum()
pg_shifts


late = -(late)-(pg_shifts)+(pg_delivered)

shifts = op_fi.shifted_revenue_usd.sum()

op1_delivery_gap = erp_rec.erp_fpa_revenue.sum() - op_fi.op_booked_revenue_usd.sum() 

variance_data = [[op_fi.op_booked_revenue_usd.sum(),'op1_booked_rev'],[erp_rec.erp_fpa_revenue.sum(),'erp_fpa_delivered'],[op1_delivery_gap,'Op1_booked:erp_fpa_delivered_gap'],['','gap_breakdown'],[early,'early'],[late,'late'],[ud,'ud'],[shifts,'shifts'],
                 [wkday_piv.wkday_revenue.sum(),'P&L_actuals'],[wkday_piv.wkday_revenue.sum() - erp_rec.erp_fpa_revenue.sum(),"P&L: erp_fpa_delivered_gap"],['','explanation_P&L: erp_fpa_delivered_gap']]
#variance_data_list = list(variance_data)


vz_explanation = vz[['p&l:delivery','wkday(invoice)_advertiser']]
vz_explanation.rename(columns = {'p&l:delivery':'revenue','wkday(invoice)_advertiser':'category'},inplace = True)
vz_explanation;

# Sales Order Line Item Shifts & Missing Delivery Category Detail

In [ ]:
#Missing Delivery category via sales line item
#toggle on/off
pd.pivot_table(op_fi, index = ['billing_month','category','sales_order_nm','sales_order_id', 'sales_order_line_item_id'],  
               values = 'missing_delivery_usd', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index().sort_values(['category','missing_delivery_usd'],ascending = False)

,billing_month,category,sales_order_nm,sales_order_id,sales_order_line_item_id,missing_delivery_usd
118,2023-04-01,UD,Home Chef_Video_Q2'23,82599,1148037,"$3,103"
119,2023-04-01,UD,Home Chef_Video_Q2'23,82599,1148038,"$2,771"
120,2023-04-01,UD,Home Chef_Video_Q2'23,82599,1148375,"$2,661"
117,2023-04-01,UD,Home Chef_Video_Q2'23,82599,1148036,"$2,495"
116,2023-04-01,UD,Home Chef_Video_Q2'23,82599,1148035,"$1,320"
110,2023-04-01,UD,HPE_FY23_Annual Unified Campaign,82452,1147284,$11
63,2023-04-01,UD,CS_Algolia_Q1_FY24_B2C Ecommerce_RedVentures,82578,1148026,$2
70,2023-04-01,UD,HPE_FY23_Annual Unified Campaign,82452,1147113,$1
81,2023-04-01,UD,HPE_FY23_Annual Unified Campaign,82452,1147172,$1
89,2023-04-01,UD,HPE_FY23_Annual Unified Campaign,82452,1147181,$1


In [ ]:
# Shifts via sales_order_line_item_id 
#Toggle on for details
pd.pivot_table(op_fi, index = ['billing_month','sales_order_nm','sales_order_id', 'sales_order_line_item_id'],  values = 'shifted_revenue_usd', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).reset_index();

# Variance Report

In [ ]:
#@title
variance_rpt = pd.DataFrame(variance_data, columns = ['revenue', 'category'])

variance_rpt.append(vz_explanation)

,revenue,category
0,"$901,948",op1_booked_rev
1,"$865,991",erp_fpa_delivered
2,"$-35,957",Op1_booked:erp_fpa_delivered_gap
3,,gap_breakdown
4,"$159,735",early
5,"$-86,432",late
6,"$-12,382",ud
7,"$-96,879",shifts
8,"$850,977",P&L_actuals
9,"$-15,014",P&L: erp_fpa_delivered_gap


In [ ]:
#drop sf_op1 piv to google sheet
xx
#paste workbook URL here
workbook = 'EOM Reporting Analysis - March 2023- A.J.'

eom_rpt = gc.open(workbook)

sfdc_v_op1_ws = eom_rpt.worksheet('SFDC_vs._Op1')

row = 2
column = 1

set_with_dataframe(sfdc_v_op1_ws,sf_op1,row,column)

NameError: ignored